In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv', sep='\t', index_col=0)
test = pd.read_csv('test.csv', sep='\t', index_col=0)

In [3]:
train = train.rename({'0': 'target'}, axis=1)
test = test.rename({'0': 'target'}, axis=1)

In [4]:
#indirectly dropping constant features, creating categorical with threshold of 10 unique values
binary = []
categorical = []
others = []
for col in train.drop(['target'], 1).columns:
    len_values = train[col].value_counts().shape[0]
    if len_values == 2:
        binary.append(col)
    elif (len_values > 2) & (len_values <= 15):
        categorical.append(col)
    elif len_values > 15:
        others.append(col)

In [5]:
features = binary + categorical + others

In [6]:
#correlation matrix
train_corr = train.corr()

In [7]:
neg_pos_train_corr = pd.DataFrame(data=np.zeros((4, len(features))), columns=features, index=['neg', 'neg_idx', 'pos', 'pos_idx'])

In [8]:
#finding the biggest and the lowest (< 0) correlations for each feature
for feature in features:
    neg_pos_train_corr.loc['neg', feature] = train_corr.loc[feature].sort_values().drop([feature]).min()
    neg_pos_train_corr.loc['neg_idx', feature] = train_corr.loc[feature].sort_values().drop([feature]).idxmin()
    neg_pos_train_corr.loc['pos', feature] = train_corr.loc[feature].sort_values().drop([feature]).max()
    neg_pos_train_corr.loc['pos_idx', feature] = train_corr.loc[feature].sort_values().drop([feature]).idxmax()

In [9]:
#highest indirectly correlated features
high_neg_corr = neg_pos_train_corr.T[(neg_pos_train_corr.loc['neg'] > -0.99) & \
                                     (neg_pos_train_corr.loc['neg'] < -0.9)][['neg', 'neg_idx']]

In [10]:
#highest directly correlated
high_pos_corr = neg_pos_train_corr.T[(neg_pos_train_corr.loc['pos'] < 0.99) & \
                                     (neg_pos_train_corr.loc['pos'] > 0.9)][['pos', 'pos_idx']]

In [11]:
high_pos_corr.sort_values(by='pos', ascending=False)

,pos,pos_idx
115,0.989875,122
122,0.989875,115
298,0.986225,335
325,0.985693,319
319,0.985693,325
35,0.966677,146
146,0.966677,35
323,0.964712,335
326,0.96018,325
39,0.955762,148


In [12]:
high_neg_corr.sort_values(by='neg')

,neg,neg_idx
322,-0.965317,326
326,-0.965317,322
299,-0.962447,331
331,-0.962447,299
325,-0.911986,322


In [13]:
f_pos_pairs = []
for idx in high_pos_corr.index:
    f_pos_pairs.append(tuple(sorted((idx, high_pos_corr.loc[idx].pos_idx))))

In [14]:
f_pos_pairs = set(f_pos_pairs)

In [15]:
f_neg_pairs = []
for idx in high_neg_corr.index:
    f_neg_pairs.append(tuple(sorted((idx, high_neg_corr.loc[idx].neg_idx))))

In [16]:
f_neg_pairs = set(f_neg_pairs)

In [17]:
new_f = []

In [18]:
#creating new features according to high correlation between some pairs of features
for i, j in f_pos_pairs:
    new_col = i+'-'+j
    train[new_col] = np.where(train[i].apply(round) == train[j].apply(round), 0, 1)
    test[new_col] = np.where(test[i].apply(round) == test[j].apply(round), 0, 1)
    new_f.append(new_col)

In [19]:
for i, j in f_neg_pairs:
    new_col = i+'-'+j
    train[new_col] = np.where(train[i].apply(round) != train[j].apply(round), 0, 1)
    test[new_col] = np.where(test[i].apply(round) != test[j].apply(round), 0, 1)
    new_f.append(new_col)

In [20]:
n_features = features + new_f

In [21]:
from sklearn.ensemble import RandomForestClassifier
feature_filter = RandomForestClassifier(
    random_state=42,
    n_estimators=10,
    n_jobs=3
)
feature_filter.fit(train[n_features], train.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=3,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [22]:
f_importance = pd.Series(data=feature_filter.feature_importances_, index=n_features)

In [23]:
f_importance.sort_values(ascending=False)

98         0.020996
102        0.020785
105        0.020044
104        0.018300
101        0.017191
201        0.016089
208        0.015091
112        0.014238
207        0.014236
115        0.014060
106        0.013919
203        0.013900
108        0.013766
204        0.013517
109        0.013401
199        0.012963
202        0.012907
107        0.012893
196        0.012829
114        0.012766
195        0.012753
205        0.012617
197        0.012605
206        0.012594
126        0.012431
125        0.012411
194        0.012403
198        0.012377
200        0.012040
209        0.011930
             ...   
182        0.000000
259        0.000000
161        0.000000
232        0.000000
130        0.000000
149        0.000000
193        0.000000
258        0.000000
170        0.000000
319-325    0.000000
3          0.000000
323-335    0.000000
139        0.000000
115-122    0.000000
191        0.000000
224        0.000000
247        0.000000
300        0.000000
301        0.000000


In [24]:
zero_importance = f_importance[f_importance == 0].index.values

In [25]:
#dropping redundant (importance = 0)
non_red_features = n_features.copy()
for elem in zero_importance:
    non_red_features.remove(elem)

In [26]:
#size of new features
print(len(features), len(n_features), len(non_red_features))

341 358 327


In [27]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(
    n_splits=8,
    shuffle=True,
    random_state=42
)

In [28]:
data = pd.concat([train, test])

In [29]:
for col in categorical:
    print(data[col].value_counts().sort_index())

0.000000        3
0.045455      375
0.090909    30822
0.136364     2931
0.181818      312
0.227273       64
0.272727       26
0.318182       96
0.363636       23
0.409091        9
0.454545        3
0.545455        1
1.000000        1
Name: 7, dtype: int64
0.00     4978
0.25       40
0.50    13054
0.75    11128
1.00     5466
Name: 117, dtype: int64
0.000000     4978
0.010753      391
0.021505     1742
0.032258     2748
0.043011      925
0.053763     5887
0.064516     3006
0.075269     2299
0.086022     1565
0.096774    11097
0.107527       28
Name: 120, dtype: int64
0.00       13
0.25      389
0.50    28345
0.75     5215
1.00      704
Name: 124, dtype: int64
0.000000      13
0.010753    3987
0.021505    8926
0.032258    7109
0.043011    1504
0.053763    6752
0.064516    1698
0.075269    1537
0.086022     591
0.096774    2549
Name: 127, dtype: int64
0.000000      206
0.003847        1
0.026054       88
0.233192     4396
1.000000    29975
Name: 212, dtype: int64
0.000000      195
0.024605

In [30]:
ordinal = ['108', '117', '120', '124', '127', '341']

In [31]:
data['341'] = data['341'].round(2)

In [32]:
categorical = [cat for cat in categorical if cat not in ordinal]

In [33]:
def ordinal_encoder(col):
    values = col.value_counts().sort_index().index.values
    new_values = list(range(len(values)))
    return dict(zip(values, new_values))

In [34]:
for feature in ordinal:
    data[feature] = data[feature].map(ordinal_encoder(data[feature]))

In [35]:
from sklearn.preprocessing import LabelEncoder
for col in categorical:
    enc = LabelEncoder()
    data[col] = enc.fit_transform(data[col])

In [36]:
new_train = data[~data.target.isna()]
new_test = data[data.target.isna()]

# 0. Baseline LGBM

In [37]:
import lightgbm as lgb

In [38]:
n_rounds = 2000
params_baseline = {
    'objective':'binary',
    'seed': 42,
    
    'learning_rate': 0.009,
    'num_threads': 3,
    'metric': 'auc',
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 1e-3,
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq': 1,
    'num_leaves': 31
}


lgb_train_baseline = lgb.Dataset(new_train[n_features], label=new_train.target, free_raw_data=False)

In [40]:
%%time
result = lgb.cv(params_baseline, lgb_train_baseline, n_rounds, folds=cv.split(new_train[n_features], new_train.target), early_stopping_rounds=100, verbose_eval=1)

[1]	cv_agg's auc: 0.674846 + 0.0136018
[2]	cv_agg's auc: 0.695462 + 0.0065697
[3]	cv_agg's auc: 0.703457 + 0.00571569
[4]	cv_agg's auc: 0.704996 + 0.00614639
[5]	cv_agg's auc: 0.707595 + 0.00692599
[6]	cv_agg's auc: 0.708677 + 0.00607066
[7]	cv_agg's auc: 0.709786 + 0.00563661
[8]	cv_agg's auc: 0.710674 + 0.00553417
[9]	cv_agg's auc: 0.71166 + 0.00502374
[10]	cv_agg's auc: 0.712032 + 0.00500985
[11]	cv_agg's auc: 0.71183 + 0.00506942
[12]	cv_agg's auc: 0.711886 + 0.00535968
[13]	cv_agg's auc: 0.711697 + 0.00542443
[14]	cv_agg's auc: 0.712151 + 0.00571014
[15]	cv_agg's auc: 0.712599 + 0.00602796
[16]	cv_agg's auc: 0.712764 + 0.00607151
[17]	cv_agg's auc: 0.712938 + 0.00615523
[18]	cv_agg's auc: 0.713388 + 0.00626132
[19]	cv_agg's auc: 0.713758 + 0.00616096
[20]	cv_agg's auc: 0.713872 + 0.00598248
[21]	cv_agg's auc: 0.714017 + 0.00623659
[22]	cv_agg's auc: 0.714386 + 0.00658715
[23]	cv_agg's auc: 0.71453 + 0.00672915
[24]	cv_agg's auc: 0.71437 + 0.00682201
[25]	cv_agg's auc: 0.714514 + 0

[200]	cv_agg's auc: 0.729288 + 0.00552582
[201]	cv_agg's auc: 0.729338 + 0.00557313
[202]	cv_agg's auc: 0.729445 + 0.00558379
[203]	cv_agg's auc: 0.729499 + 0.00561088
[204]	cv_agg's auc: 0.729601 + 0.00560901
[205]	cv_agg's auc: 0.729645 + 0.00556568
[206]	cv_agg's auc: 0.729742 + 0.00559051
[207]	cv_agg's auc: 0.729891 + 0.0055466
[208]	cv_agg's auc: 0.729996 + 0.00544738
[209]	cv_agg's auc: 0.730011 + 0.00545268
[210]	cv_agg's auc: 0.73012 + 0.00546567
[211]	cv_agg's auc: 0.730185 + 0.00550627
[212]	cv_agg's auc: 0.730255 + 0.00548322
[213]	cv_agg's auc: 0.730317 + 0.00551614
[214]	cv_agg's auc: 0.730367 + 0.00548037
[215]	cv_agg's auc: 0.73052 + 0.00543997
[216]	cv_agg's auc: 0.730613 + 0.00547278
[217]	cv_agg's auc: 0.730662 + 0.00545578
[218]	cv_agg's auc: 0.730728 + 0.00540273
[219]	cv_agg's auc: 0.730799 + 0.00538985
[220]	cv_agg's auc: 0.730831 + 0.00539729
[221]	cv_agg's auc: 0.730955 + 0.00537978
[222]	cv_agg's auc: 0.731044 + 0.00535325
[223]	cv_agg's auc: 0.731135 + 0.0053

[397]	cv_agg's auc: 0.739722 + 0.0046364
[398]	cv_agg's auc: 0.739774 + 0.0046558
[399]	cv_agg's auc: 0.739799 + 0.00467445
[400]	cv_agg's auc: 0.739826 + 0.00465805
[401]	cv_agg's auc: 0.739859 + 0.00465276
[402]	cv_agg's auc: 0.739859 + 0.00466106
[403]	cv_agg's auc: 0.739898 + 0.00467785
[404]	cv_agg's auc: 0.739921 + 0.00466908
[405]	cv_agg's auc: 0.739958 + 0.00462938
[406]	cv_agg's auc: 0.740003 + 0.00461769
[407]	cv_agg's auc: 0.74002 + 0.00461542
[408]	cv_agg's auc: 0.740001 + 0.00462558
[409]	cv_agg's auc: 0.740077 + 0.00463111
[410]	cv_agg's auc: 0.740097 + 0.00463811
[411]	cv_agg's auc: 0.740148 + 0.00465202
[412]	cv_agg's auc: 0.74017 + 0.00466088
[413]	cv_agg's auc: 0.740207 + 0.00467649
[414]	cv_agg's auc: 0.740225 + 0.00467042
[415]	cv_agg's auc: 0.740275 + 0.00470354
[416]	cv_agg's auc: 0.740316 + 0.0046877
[417]	cv_agg's auc: 0.74034 + 0.00470906
[418]	cv_agg's auc: 0.740347 + 0.00473479
[419]	cv_agg's auc: 0.740368 + 0.00473491
[420]	cv_agg's auc: 0.74039 + 0.00471052

[594]	cv_agg's auc: 0.743591 + 0.00421497
[595]	cv_agg's auc: 0.743622 + 0.00420433
[596]	cv_agg's auc: 0.743631 + 0.00421327
[597]	cv_agg's auc: 0.74363 + 0.00421655
[598]	cv_agg's auc: 0.743631 + 0.00422496
[599]	cv_agg's auc: 0.743653 + 0.00423847
[600]	cv_agg's auc: 0.743658 + 0.0042469
[601]	cv_agg's auc: 0.743692 + 0.00423075
[602]	cv_agg's auc: 0.743722 + 0.00427476
[603]	cv_agg's auc: 0.743741 + 0.00427539
[604]	cv_agg's auc: 0.743743 + 0.00426069
[605]	cv_agg's auc: 0.743765 + 0.00425256
[606]	cv_agg's auc: 0.743764 + 0.00425502
[607]	cv_agg's auc: 0.743784 + 0.00429566
[608]	cv_agg's auc: 0.743793 + 0.00429607
[609]	cv_agg's auc: 0.743791 + 0.00429386
[610]	cv_agg's auc: 0.743804 + 0.00428769
[611]	cv_agg's auc: 0.743815 + 0.00428374
[612]	cv_agg's auc: 0.743828 + 0.00427059
[613]	cv_agg's auc: 0.743836 + 0.00426566
[614]	cv_agg's auc: 0.743855 + 0.00425058
[615]	cv_agg's auc: 0.743853 + 0.00424436
[616]	cv_agg's auc: 0.74383 + 0.00425206
[617]	cv_agg's auc: 0.743811 + 0.0042

[791]	cv_agg's auc: 0.745297 + 0.00421138
[792]	cv_agg's auc: 0.745292 + 0.00420826
[793]	cv_agg's auc: 0.745289 + 0.00421308
[794]	cv_agg's auc: 0.745313 + 0.00421478
[795]	cv_agg's auc: 0.745334 + 0.00422112
[796]	cv_agg's auc: 0.745351 + 0.00420207
[797]	cv_agg's auc: 0.745319 + 0.0042077
[798]	cv_agg's auc: 0.745301 + 0.00419573
[799]	cv_agg's auc: 0.745324 + 0.00418343
[800]	cv_agg's auc: 0.745318 + 0.00419211
[801]	cv_agg's auc: 0.74534 + 0.0042151
[802]	cv_agg's auc: 0.74532 + 0.00424987
[803]	cv_agg's auc: 0.74531 + 0.0042304
[804]	cv_agg's auc: 0.745313 + 0.0042078
[805]	cv_agg's auc: 0.745314 + 0.004205
[806]	cv_agg's auc: 0.745339 + 0.00418582
[807]	cv_agg's auc: 0.745354 + 0.00418591
[808]	cv_agg's auc: 0.745353 + 0.00418947
[809]	cv_agg's auc: 0.745342 + 0.00422287
[810]	cv_agg's auc: 0.745374 + 0.00424547
[811]	cv_agg's auc: 0.745383 + 0.00426387
[812]	cv_agg's auc: 0.74538 + 0.00427106
[813]	cv_agg's auc: 0.745396 + 0.00424848
[814]	cv_agg's auc: 0.745426 + 0.00424152
[8

[988]	cv_agg's auc: 0.746107 + 0.0042805
[989]	cv_agg's auc: 0.746121 + 0.00426926
[990]	cv_agg's auc: 0.746118 + 0.00425194
[991]	cv_agg's auc: 0.746126 + 0.00425432
[992]	cv_agg's auc: 0.746118 + 0.00427856
[993]	cv_agg's auc: 0.746119 + 0.00428799
[994]	cv_agg's auc: 0.746117 + 0.00428116
[995]	cv_agg's auc: 0.746124 + 0.0042666
[996]	cv_agg's auc: 0.746141 + 0.00425958
[997]	cv_agg's auc: 0.746153 + 0.0042775
[998]	cv_agg's auc: 0.746157 + 0.00427113
[999]	cv_agg's auc: 0.746145 + 0.0042873
[1000]	cv_agg's auc: 0.746159 + 0.00424648
[1001]	cv_agg's auc: 0.746147 + 0.00425691
[1002]	cv_agg's auc: 0.746155 + 0.00424918
[1003]	cv_agg's auc: 0.746151 + 0.00426053
[1004]	cv_agg's auc: 0.746158 + 0.00428072
[1005]	cv_agg's auc: 0.746176 + 0.00428664
[1006]	cv_agg's auc: 0.746166 + 0.00428281
[1007]	cv_agg's auc: 0.74616 + 0.00428827
[1008]	cv_agg's auc: 0.746178 + 0.00430974
[1009]	cv_agg's auc: 0.746167 + 0.0043097
[1010]	cv_agg's auc: 0.746192 + 0.00431447
[1011]	cv_agg's auc: 0.746211

[1180]	cv_agg's auc: 0.746698 + 0.00410153
[1181]	cv_agg's auc: 0.746702 + 0.0041228
[1182]	cv_agg's auc: 0.746691 + 0.00413421
[1183]	cv_agg's auc: 0.74668 + 0.00414068
[1184]	cv_agg's auc: 0.746701 + 0.00413101
[1185]	cv_agg's auc: 0.746697 + 0.00413224
[1186]	cv_agg's auc: 0.746722 + 0.00411517
[1187]	cv_agg's auc: 0.746723 + 0.00413204
[1188]	cv_agg's auc: 0.746719 + 0.00411204
[1189]	cv_agg's auc: 0.746732 + 0.00411727
[1190]	cv_agg's auc: 0.746724 + 0.00412982
[1191]	cv_agg's auc: 0.746725 + 0.00412781
[1192]	cv_agg's auc: 0.746717 + 0.00414204
[1193]	cv_agg's auc: 0.746722 + 0.0041167
[1194]	cv_agg's auc: 0.746713 + 0.00412443
[1195]	cv_agg's auc: 0.746735 + 0.00412666
[1196]	cv_agg's auc: 0.746754 + 0.00411951
[1197]	cv_agg's auc: 0.746756 + 0.0041286
[1198]	cv_agg's auc: 0.746748 + 0.00411059
[1199]	cv_agg's auc: 0.746759 + 0.00410115
[1200]	cv_agg's auc: 0.746746 + 0.00407086
[1201]	cv_agg's auc: 0.74676 + 0.00408651
[1202]	cv_agg's auc: 0.746771 + 0.00408234
[1203]	cv_agg's 

[1372]	cv_agg's auc: 0.746927 + 0.00408476
[1373]	cv_agg's auc: 0.746934 + 0.00408622
[1374]	cv_agg's auc: 0.746944 + 0.0040745
[1375]	cv_agg's auc: 0.746944 + 0.00407841
[1376]	cv_agg's auc: 0.746948 + 0.00408386
[1377]	cv_agg's auc: 0.746933 + 0.00409052
[1378]	cv_agg's auc: 0.746943 + 0.00410093
[1379]	cv_agg's auc: 0.746957 + 0.00412807
[1380]	cv_agg's auc: 0.746939 + 0.00415629
[1381]	cv_agg's auc: 0.746941 + 0.00415061
[1382]	cv_agg's auc: 0.746939 + 0.00413444
[1383]	cv_agg's auc: 0.746939 + 0.0041357
[1384]	cv_agg's auc: 0.746939 + 0.00414244
[1385]	cv_agg's auc: 0.746941 + 0.00411307
[1386]	cv_agg's auc: 0.746958 + 0.00412849
[1387]	cv_agg's auc: 0.746938 + 0.00413566
[1388]	cv_agg's auc: 0.746941 + 0.0041399
[1389]	cv_agg's auc: 0.746938 + 0.00414412
[1390]	cv_agg's auc: 0.74694 + 0.00416162
[1391]	cv_agg's auc: 0.746934 + 0.00414717
[1392]	cv_agg's auc: 0.746935 + 0.00415638
[1393]	cv_agg's auc: 0.74695 + 0.00417511
[1394]	cv_agg's auc: 0.746955 + 0.00418449
[1395]	cv_agg's 

In [41]:
from sklearn.metrics import roc_auc_score
def check_train_score(p, lgb_data, data, target, kf, num_rounds):
    roc_auc = []
    for train, val in kf.split(data,target):
        temp_lgb_train = lgb_data.subset(train)
        temp_lgb_val = lgb_data.subset(val)
        temp_model = lgb.train(p, temp_lgb_train, num_rounds, verbose_eval=num_rounds)
        roc_auc.append([roc_auc_score(target[train],temp_model.predict(data.loc[train])), roc_auc_score(target[val],temp_model.predict(data.loc[val]))])
    return np.mean(roc_auc, axis=0)

check_train_score(params_baseline, lgb_train_baseline, new_train[n_features], new_train.target, cv, 1440)

array([0.91021915, 0.74702547])

# 1. UMAP and clustering

In [42]:
import umap
from sklearn.cluster import DBSCAN

In [43]:
%%time
umap_res = umap.UMAP(n_neighbors=10,
                     min_dist=0.3,
                     #metric='correlation',
                     random_state=42).fit_transform(data[n_features])

Wall time: 2min 51s


In [44]:
umap_data = pd.DataFrame(data=umap_res, index=data.index)

In [45]:
%%time
findcl = DBSCAN(
    eps=0.5,
    min_samples=100,
    n_jobs=3
)
clusters = findcl.fit_predict(umap_data.values)

Wall time: 1.24 s


In [46]:
np.unique(clusters)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int64)

In [47]:
pd.Series(clusters).value_counts()

 3     6688
 5     4579
 0     3218
 10    3142
 1     2330
 2     2222
 13    2017
 4     1398
 6      970
 9      921
 8      728
-1      668
 12     635
 7      620
 16     589
 18     520
 20     510
 11     480
 19     452
 23     306
 17     289
 14     221
 27     194
 22     190
 15     182
 26     158
 25     157
 21     144
 24     138
dtype: int64

In [48]:
target_cluster = pd.DataFrame(np.zeros((data.shape[0], 2)), index=data.index)
target_cluster.columns = ['target', 'cluster']
target_cluster['target'] = data.target
target_cluster['cluster'] = clusters

In [49]:
for cl in np.unique(clusters):
    print(
        'In {} cluster {:.2f}% of clients are users of service'.format(cl, 
                                                        target_cluster[target_cluster.cluster == cl]['target'].mean()*100))

In -1 cluster 11.51% of clients are users of service
In 0 cluster 30.51% of clients are users of service
In 1 cluster 24.73% of clients are users of service
In 2 cluster 20.83% of clients are users of service
In 3 cluster 11.08% of clients are users of service
In 4 cluster 14.60% of clients are users of service
In 5 cluster 12.38% of clients are users of service
In 6 cluster 10.47% of clients are users of service
In 7 cluster 23.45% of clients are users of service
In 8 cluster 22.71% of clients are users of service
In 9 cluster 21.10% of clients are users of service
In 10 cluster 23.16% of clients are users of service
In 11 cluster 19.53% of clients are users of service
In 12 cluster 15.25% of clients are users of service
In 13 cluster 18.05% of clients are users of service
In 14 cluster 15.23% of clients are users of service
In 15 cluster 19.28% of clients are users of service
In 16 cluster 16.57% of clients are users of service
In 17 cluster 10.23% of clients are users of service
In 

In [50]:
data['cluster'] = clusters

In [51]:
new_train = data[~data.target.isna()]
new_test = data[data.target.isna()]

In [52]:
features_cl = n_features + ['cluster']

In [70]:
params_cl = {
    'objective':'binary',
    'seed': 42,
    
    'learning_rate': 0.009,
    'num_threads': 3,
    'metric': 'auc',
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 1e-3,
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq': 1,
    'num_leaves': 31
}


lgb_train_cl = lgb.Dataset(new_train[features_cl], label=new_train.target, free_raw_data=False)

In [71]:
%%time
result = lgb.cv(params_cl, lgb_train_cl, n_rounds, folds=cv.split(new_train[features_cl], new_train.target), early_stopping_rounds=100, verbose_eval=1)

[1]	cv_agg's auc: 0.674412 + 0.013834
[2]	cv_agg's auc: 0.694692 + 0.00949113
[3]	cv_agg's auc: 0.701541 + 0.00663312
[4]	cv_agg's auc: 0.704573 + 0.00719343
[5]	cv_agg's auc: 0.705689 + 0.00686909
[6]	cv_agg's auc: 0.70742 + 0.0069632
[7]	cv_agg's auc: 0.708859 + 0.00709336
[8]	cv_agg's auc: 0.709103 + 0.00756751
[9]	cv_agg's auc: 0.709974 + 0.00854448
[10]	cv_agg's auc: 0.711567 + 0.00831697
[11]	cv_agg's auc: 0.712155 + 0.00846022
[12]	cv_agg's auc: 0.712353 + 0.00821396
[13]	cv_agg's auc: 0.712307 + 0.0082542
[14]	cv_agg's auc: 0.712426 + 0.00791127
[15]	cv_agg's auc: 0.71266 + 0.00801492
[16]	cv_agg's auc: 0.713012 + 0.0076853
[17]	cv_agg's auc: 0.713523 + 0.00738815
[18]	cv_agg's auc: 0.713442 + 0.00740416
[19]	cv_agg's auc: 0.713723 + 0.0075687
[20]	cv_agg's auc: 0.713958 + 0.00751414
[21]	cv_agg's auc: 0.713895 + 0.00757448
[22]	cv_agg's auc: 0.714269 + 0.00760127
[23]	cv_agg's auc: 0.714451 + 0.00768081
[24]	cv_agg's auc: 0.714351 + 0.00767642
[25]	cv_agg's auc: 0.71438 + 0.00

[201]	cv_agg's auc: 0.729997 + 0.00537434
[202]	cv_agg's auc: 0.730084 + 0.00543988
[203]	cv_agg's auc: 0.730125 + 0.00542693
[204]	cv_agg's auc: 0.730172 + 0.00538615
[205]	cv_agg's auc: 0.73027 + 0.00536462
[206]	cv_agg's auc: 0.730316 + 0.0053766
[207]	cv_agg's auc: 0.730426 + 0.00540311
[208]	cv_agg's auc: 0.73051 + 0.00538069
[209]	cv_agg's auc: 0.730534 + 0.00535775
[210]	cv_agg's auc: 0.730594 + 0.00535614
[211]	cv_agg's auc: 0.730718 + 0.00533815
[212]	cv_agg's auc: 0.730771 + 0.00534351
[213]	cv_agg's auc: 0.730867 + 0.00530467
[214]	cv_agg's auc: 0.73094 + 0.00529413
[215]	cv_agg's auc: 0.731033 + 0.00529468
[216]	cv_agg's auc: 0.731107 + 0.00527568
[217]	cv_agg's auc: 0.731139 + 0.00529575
[218]	cv_agg's auc: 0.731253 + 0.00530711
[219]	cv_agg's auc: 0.731323 + 0.00529877
[220]	cv_agg's auc: 0.731419 + 0.00527134
[221]	cv_agg's auc: 0.731512 + 0.00528415
[222]	cv_agg's auc: 0.73155 + 0.00527401
[223]	cv_agg's auc: 0.731612 + 0.00530091
[224]	cv_agg's auc: 0.731676 + 0.005253

[397]	cv_agg's auc: 0.740092 + 0.00448399
[398]	cv_agg's auc: 0.740116 + 0.00446775
[399]	cv_agg's auc: 0.740141 + 0.0044946
[400]	cv_agg's auc: 0.740172 + 0.00450418
[401]	cv_agg's auc: 0.740197 + 0.00447124
[402]	cv_agg's auc: 0.740241 + 0.00452095
[403]	cv_agg's auc: 0.740282 + 0.00453735
[404]	cv_agg's auc: 0.740293 + 0.00451634
[405]	cv_agg's auc: 0.740345 + 0.00451033
[406]	cv_agg's auc: 0.74037 + 0.00452726
[407]	cv_agg's auc: 0.740406 + 0.00453551
[408]	cv_agg's auc: 0.740411 + 0.00452212
[409]	cv_agg's auc: 0.740409 + 0.00452999
[410]	cv_agg's auc: 0.740433 + 0.00450417
[411]	cv_agg's auc: 0.740446 + 0.00449982
[412]	cv_agg's auc: 0.740457 + 0.00449517
[413]	cv_agg's auc: 0.740492 + 0.00450196
[414]	cv_agg's auc: 0.740508 + 0.00449072
[415]	cv_agg's auc: 0.740507 + 0.00450316
[416]	cv_agg's auc: 0.74056 + 0.0045104
[417]	cv_agg's auc: 0.740602 + 0.00450951
[418]	cv_agg's auc: 0.740636 + 0.00451463
[419]	cv_agg's auc: 0.740614 + 0.00450117
[420]	cv_agg's auc: 0.740656 + 0.00445

[594]	cv_agg's auc: 0.743446 + 0.00401614
[595]	cv_agg's auc: 0.743469 + 0.00400009
[596]	cv_agg's auc: 0.743465 + 0.00399445
[597]	cv_agg's auc: 0.743492 + 0.00399705
[598]	cv_agg's auc: 0.74349 + 0.00400119
[599]	cv_agg's auc: 0.743522 + 0.0040057
[600]	cv_agg's auc: 0.743516 + 0.00400565
[601]	cv_agg's auc: 0.743532 + 0.00399135
[602]	cv_agg's auc: 0.743536 + 0.00400238
[603]	cv_agg's auc: 0.743544 + 0.00397916
[604]	cv_agg's auc: 0.743575 + 0.00398216
[605]	cv_agg's auc: 0.743583 + 0.00396571
[606]	cv_agg's auc: 0.743583 + 0.00398171
[607]	cv_agg's auc: 0.743561 + 0.0039835
[608]	cv_agg's auc: 0.74361 + 0.00397152
[609]	cv_agg's auc: 0.743642 + 0.00399095
[610]	cv_agg's auc: 0.743681 + 0.00399207
[611]	cv_agg's auc: 0.743678 + 0.0039893
[612]	cv_agg's auc: 0.743671 + 0.00400645
[613]	cv_agg's auc: 0.743716 + 0.00399982
[614]	cv_agg's auc: 0.743725 + 0.00399441
[615]	cv_agg's auc: 0.74376 + 0.00398592
[616]	cv_agg's auc: 0.743775 + 0.00398979
[617]	cv_agg's auc: 0.743801 + 0.0040043

[792]	cv_agg's auc: 0.745197 + 0.00377896
[793]	cv_agg's auc: 0.745177 + 0.00380184
[794]	cv_agg's auc: 0.745171 + 0.00381032
[795]	cv_agg's auc: 0.745163 + 0.00380376
[796]	cv_agg's auc: 0.74517 + 0.00377905
[797]	cv_agg's auc: 0.745147 + 0.00378061
[798]	cv_agg's auc: 0.745149 + 0.00377146
[799]	cv_agg's auc: 0.745154 + 0.00377384
[800]	cv_agg's auc: 0.745184 + 0.00379488
[801]	cv_agg's auc: 0.745175 + 0.00379184
[802]	cv_agg's auc: 0.745164 + 0.0038042
[803]	cv_agg's auc: 0.745169 + 0.00380145
[804]	cv_agg's auc: 0.745162 + 0.00378336
[805]	cv_agg's auc: 0.745173 + 0.0037943
[806]	cv_agg's auc: 0.745177 + 0.00377866
[807]	cv_agg's auc: 0.745177 + 0.00376599
[808]	cv_agg's auc: 0.745169 + 0.00376781
[809]	cv_agg's auc: 0.74518 + 0.00378166
[810]	cv_agg's auc: 0.745182 + 0.00379476
[811]	cv_agg's auc: 0.745189 + 0.00380958
[812]	cv_agg's auc: 0.745192 + 0.00379268
[813]	cv_agg's auc: 0.745224 + 0.00379632
[814]	cv_agg's auc: 0.745218 + 0.00380006
[815]	cv_agg's auc: 0.745247 + 0.00380

[989]	cv_agg's auc: 0.745893 + 0.00342718
[990]	cv_agg's auc: 0.745913 + 0.0034245
[991]	cv_agg's auc: 0.745914 + 0.00342475
[992]	cv_agg's auc: 0.745931 + 0.00345638
[993]	cv_agg's auc: 0.745933 + 0.00345767
[994]	cv_agg's auc: 0.745915 + 0.00345976
[995]	cv_agg's auc: 0.745916 + 0.00347095
[996]	cv_agg's auc: 0.745928 + 0.00346587
[997]	cv_agg's auc: 0.74592 + 0.00344701
[998]	cv_agg's auc: 0.745918 + 0.00346415
[999]	cv_agg's auc: 0.745924 + 0.00345424
[1000]	cv_agg's auc: 0.745923 + 0.00347806
[1001]	cv_agg's auc: 0.745913 + 0.00347247
[1002]	cv_agg's auc: 0.745929 + 0.00345871
[1003]	cv_agg's auc: 0.74594 + 0.00346635
[1004]	cv_agg's auc: 0.745936 + 0.00348045
[1005]	cv_agg's auc: 0.745933 + 0.00348376
[1006]	cv_agg's auc: 0.74596 + 0.00350415
[1007]	cv_agg's auc: 0.745962 + 0.00349587
[1008]	cv_agg's auc: 0.745952 + 0.00349505
[1009]	cv_agg's auc: 0.745955 + 0.00350334
[1010]	cv_agg's auc: 0.745964 + 0.00350267
[1011]	cv_agg's auc: 0.74598 + 0.00349934
[1012]	cv_agg's auc: 0.7459

[1181]	cv_agg's auc: 0.746491 + 0.00359165
[1182]	cv_agg's auc: 0.746504 + 0.00359113
[1183]	cv_agg's auc: 0.746506 + 0.00358817
[1184]	cv_agg's auc: 0.746516 + 0.00359041
[1185]	cv_agg's auc: 0.746522 + 0.00361323
[1186]	cv_agg's auc: 0.746531 + 0.00361096
[1187]	cv_agg's auc: 0.746552 + 0.00360871
[1188]	cv_agg's auc: 0.746562 + 0.00359638
[1189]	cv_agg's auc: 0.74654 + 0.00358349
[1190]	cv_agg's auc: 0.746528 + 0.00360286
[1191]	cv_agg's auc: 0.746544 + 0.00359059
[1192]	cv_agg's auc: 0.746531 + 0.00360454
[1193]	cv_agg's auc: 0.746543 + 0.00360778
[1194]	cv_agg's auc: 0.746554 + 0.00357519
[1195]	cv_agg's auc: 0.746548 + 0.00358395
[1196]	cv_agg's auc: 0.746545 + 0.00355491
[1197]	cv_agg's auc: 0.746539 + 0.00355888
[1198]	cv_agg's auc: 0.746539 + 0.00355781
[1199]	cv_agg's auc: 0.74657 + 0.00355791
[1200]	cv_agg's auc: 0.746555 + 0.00356452
[1201]	cv_agg's auc: 0.74657 + 0.00356397
[1202]	cv_agg's auc: 0.746568 + 0.00358593
[1203]	cv_agg's auc: 0.746569 + 0.00358506
[1204]	cv_agg'

In [72]:
check_train_score(params_cl, lgb_train_cl, new_train[features_cl], new_train.target, cv, 1237)

array([0.89610037, 0.74666292])

# 2. Clustering

In [73]:
%%time
findcl_full = DBSCAN(
    eps=3.5,
    min_samples=100,
    n_jobs=3
)
clusters_full = findcl_full.fit_predict(data[n_features].values)

Wall time: 5min 14s


In [74]:
np.unique(clusters_full)

array([-1,  0,  1,  2,  3,  4,  5,  6], dtype=int64)

In [75]:
pd.Series(clusters_full).value_counts()

-1    24815
 1     6765
 0     1475
 2      642
 4      354
 6      285
 3      177
 5      153
dtype: int64

In [76]:
target_cluster_full = pd.DataFrame(np.zeros((data.shape[0], 2)), index=data.index)
target_cluster_full.columns = ['target', 'cluster']
target_cluster_full['target'] = data.target
target_cluster_full['cluster'] = clusters_full

In [77]:
for cl in np.unique(clusters_full):
    print(
        'In {} cluster {:.2f}% of clients are users of service'.format(cl, 
                                            target_cluster_full[target_cluster_full.cluster == cl]['target'].mean()*100))

In -1 cluster 19.15% of clients are users of service
In 0 cluster 10.62% of clients are users of service
In 1 cluster 12.32% of clients are users of service
In 2 cluster 25.31% of clients are users of service
In 3 cluster 20.38% of clients are users of service
In 4 cluster 17.59% of clients are users of service
In 5 cluster 13.14% of clients are users of service
In 6 cluster 50.20% of clients are users of service


In [78]:
data['cluster_db'] = clusters_full

In [79]:
new_train = data[~data.target.isna()]
new_test = data[data.target.isna()]

In [80]:
features_db = n_features + ['cluster_db']

In [81]:
params_db = {
    'objective':'binary',
    'seed': 42,
    
    'learning_rate': 0.009,
    'num_threads': 3,
    'metric': 'auc',
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 1e-3,
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq': 1,
    'num_leaves': 31
}


lgb_train_db = lgb.Dataset(new_train[features_db], label=new_train.target, free_raw_data=False)

In [82]:
%%time
result = lgb.cv(params_db, lgb_train_db, n_rounds, folds=cv.split(new_train[features_db], new_train.target), early_stopping_rounds=100, verbose_eval=1)

[1]	cv_agg's auc: 0.674412 + 0.013834
[2]	cv_agg's auc: 0.694555 + 0.00942373
[3]	cv_agg's auc: 0.701479 + 0.00661231
[4]	cv_agg's auc: 0.704494 + 0.00722377
[5]	cv_agg's auc: 0.70593 + 0.00725872
[6]	cv_agg's auc: 0.70764 + 0.0073414
[7]	cv_agg's auc: 0.709022 + 0.0074016
[8]	cv_agg's auc: 0.709206 + 0.00787072
[9]	cv_agg's auc: 0.710025 + 0.00875668
[10]	cv_agg's auc: 0.711693 + 0.00859406
[11]	cv_agg's auc: 0.712289 + 0.00864083
[12]	cv_agg's auc: 0.712376 + 0.00838164
[13]	cv_agg's auc: 0.712308 + 0.00825108
[14]	cv_agg's auc: 0.712451 + 0.00800235
[15]	cv_agg's auc: 0.712686 + 0.00790748
[16]	cv_agg's auc: 0.713114 + 0.00764531
[17]	cv_agg's auc: 0.713591 + 0.00734974
[18]	cv_agg's auc: 0.713522 + 0.00736521
[19]	cv_agg's auc: 0.713807 + 0.00751977
[20]	cv_agg's auc: 0.714167 + 0.00758982
[21]	cv_agg's auc: 0.714014 + 0.00762755
[22]	cv_agg's auc: 0.714322 + 0.00757296
[23]	cv_agg's auc: 0.714512 + 0.0076955
[24]	cv_agg's auc: 0.714487 + 0.00765489
[25]	cv_agg's auc: 0.714488 + 0.

[201]	cv_agg's auc: 0.729984 + 0.00528569
[202]	cv_agg's auc: 0.730055 + 0.00531036
[203]	cv_agg's auc: 0.73011 + 0.00527431
[204]	cv_agg's auc: 0.730144 + 0.00523349
[205]	cv_agg's auc: 0.730248 + 0.00521918
[206]	cv_agg's auc: 0.730286 + 0.00522594
[207]	cv_agg's auc: 0.730385 + 0.00523791
[208]	cv_agg's auc: 0.730479 + 0.00522429
[209]	cv_agg's auc: 0.730488 + 0.00520928
[210]	cv_agg's auc: 0.730548 + 0.00523703
[211]	cv_agg's auc: 0.730638 + 0.00520746
[212]	cv_agg's auc: 0.730693 + 0.00522572
[213]	cv_agg's auc: 0.730762 + 0.00517811
[214]	cv_agg's auc: 0.730843 + 0.00517964
[215]	cv_agg's auc: 0.730929 + 0.00519337
[216]	cv_agg's auc: 0.73099 + 0.00518587
[217]	cv_agg's auc: 0.731012 + 0.00519977
[218]	cv_agg's auc: 0.731129 + 0.00521517
[219]	cv_agg's auc: 0.731178 + 0.00521303
[220]	cv_agg's auc: 0.731284 + 0.00517574
[221]	cv_agg's auc: 0.731362 + 0.00516621
[222]	cv_agg's auc: 0.731408 + 0.00515545
[223]	cv_agg's auc: 0.731487 + 0.00515039
[224]	cv_agg's auc: 0.731548 + 0.005

[399]	cv_agg's auc: 0.740071 + 0.00459014
[400]	cv_agg's auc: 0.74009 + 0.00460258
[401]	cv_agg's auc: 0.740119 + 0.0045503
[402]	cv_agg's auc: 0.740188 + 0.00459794
[403]	cv_agg's auc: 0.740225 + 0.0046171
[404]	cv_agg's auc: 0.740211 + 0.00461939
[405]	cv_agg's auc: 0.740264 + 0.00460477
[406]	cv_agg's auc: 0.740287 + 0.00460799
[407]	cv_agg's auc: 0.740323 + 0.00461514
[408]	cv_agg's auc: 0.740328 + 0.00457832
[409]	cv_agg's auc: 0.740319 + 0.00461751
[410]	cv_agg's auc: 0.74035 + 0.00461059
[411]	cv_agg's auc: 0.740354 + 0.00461305
[412]	cv_agg's auc: 0.740351 + 0.00462192
[413]	cv_agg's auc: 0.740368 + 0.00459492
[414]	cv_agg's auc: 0.740386 + 0.0045845
[415]	cv_agg's auc: 0.740427 + 0.00457176
[416]	cv_agg's auc: 0.740481 + 0.00458693
[417]	cv_agg's auc: 0.740525 + 0.00462278
[418]	cv_agg's auc: 0.74055 + 0.00464109
[419]	cv_agg's auc: 0.740521 + 0.00463758
[420]	cv_agg's auc: 0.740561 + 0.00462697
[421]	cv_agg's auc: 0.740591 + 0.00461244
[422]	cv_agg's auc: 0.740594 + 0.0045813

[595]	cv_agg's auc: 0.743401 + 0.00408665
[596]	cv_agg's auc: 0.743427 + 0.00408092
[597]	cv_agg's auc: 0.743437 + 0.00408466
[598]	cv_agg's auc: 0.743436 + 0.00408448
[599]	cv_agg's auc: 0.743485 + 0.00407762
[600]	cv_agg's auc: 0.743489 + 0.00407133
[601]	cv_agg's auc: 0.743494 + 0.00406876
[602]	cv_agg's auc: 0.743515 + 0.0040884
[603]	cv_agg's auc: 0.743535 + 0.0040631
[604]	cv_agg's auc: 0.743568 + 0.00406324
[605]	cv_agg's auc: 0.743554 + 0.00406985
[606]	cv_agg's auc: 0.743562 + 0.00407882
[607]	cv_agg's auc: 0.743574 + 0.00407342
[608]	cv_agg's auc: 0.743595 + 0.00405572
[609]	cv_agg's auc: 0.743603 + 0.00405946
[610]	cv_agg's auc: 0.743641 + 0.00406918
[611]	cv_agg's auc: 0.743644 + 0.00407853
[612]	cv_agg's auc: 0.74365 + 0.00409322
[613]	cv_agg's auc: 0.743675 + 0.00407222
[614]	cv_agg's auc: 0.743678 + 0.00405572
[615]	cv_agg's auc: 0.743706 + 0.00406153
[616]	cv_agg's auc: 0.743728 + 0.00410878
[617]	cv_agg's auc: 0.743768 + 0.00412402
[618]	cv_agg's auc: 0.743775 + 0.0041

[793]	cv_agg's auc: 0.745256 + 0.00383345
[794]	cv_agg's auc: 0.745262 + 0.00383552
[795]	cv_agg's auc: 0.74523 + 0.00381787
[796]	cv_agg's auc: 0.745249 + 0.00378793
[797]	cv_agg's auc: 0.745228 + 0.00379148
[798]	cv_agg's auc: 0.745254 + 0.00378904
[799]	cv_agg's auc: 0.745246 + 0.00380223
[800]	cv_agg's auc: 0.745273 + 0.00382326
[801]	cv_agg's auc: 0.74527 + 0.00381946
[802]	cv_agg's auc: 0.745276 + 0.00383475
[803]	cv_agg's auc: 0.745287 + 0.00382486
[804]	cv_agg's auc: 0.745273 + 0.00381882
[805]	cv_agg's auc: 0.745302 + 0.00381257
[806]	cv_agg's auc: 0.745301 + 0.00381795
[807]	cv_agg's auc: 0.745305 + 0.00381821
[808]	cv_agg's auc: 0.745296 + 0.00381156
[809]	cv_agg's auc: 0.74531 + 0.00383498
[810]	cv_agg's auc: 0.74533 + 0.00385649
[811]	cv_agg's auc: 0.745331 + 0.00387837
[812]	cv_agg's auc: 0.745325 + 0.00387381
[813]	cv_agg's auc: 0.745346 + 0.00387397
[814]	cv_agg's auc: 0.745351 + 0.00387069
[815]	cv_agg's auc: 0.74539 + 0.00386173
[816]	cv_agg's auc: 0.745429 + 0.003880

[989]	cv_agg's auc: 0.746119 + 0.00383289
[990]	cv_agg's auc: 0.74613 + 0.00384887
[991]	cv_agg's auc: 0.746131 + 0.00383785
[992]	cv_agg's auc: 0.746132 + 0.00385822
[993]	cv_agg's auc: 0.746134 + 0.00386424
[994]	cv_agg's auc: 0.746117 + 0.00385798
[995]	cv_agg's auc: 0.746096 + 0.0038595
[996]	cv_agg's auc: 0.746103 + 0.00387574
[997]	cv_agg's auc: 0.746099 + 0.00386809
[998]	cv_agg's auc: 0.746102 + 0.00387562
[999]	cv_agg's auc: 0.746104 + 0.0038794
[1000]	cv_agg's auc: 0.746104 + 0.00390982
[1001]	cv_agg's auc: 0.74612 + 0.0039046
[1002]	cv_agg's auc: 0.746127 + 0.00390093
[1003]	cv_agg's auc: 0.746134 + 0.00391913
[1004]	cv_agg's auc: 0.746137 + 0.00393837
[1005]	cv_agg's auc: 0.746127 + 0.00395399
[1006]	cv_agg's auc: 0.746126 + 0.00395013
[1007]	cv_agg's auc: 0.746121 + 0.00394246
[1008]	cv_agg's auc: 0.746126 + 0.00394268
[1009]	cv_agg's auc: 0.74613 + 0.00395186
[1010]	cv_agg's auc: 0.74615 + 0.00394183
[1011]	cv_agg's auc: 0.746171 + 0.00395144
[1012]	cv_agg's auc: 0.746176

[1181]	cv_agg's auc: 0.746649 + 0.00402711
[1182]	cv_agg's auc: 0.746665 + 0.00401789
[1183]	cv_agg's auc: 0.74665 + 0.00401192
[1184]	cv_agg's auc: 0.746684 + 0.00402607
[1185]	cv_agg's auc: 0.746692 + 0.00404539
[1186]	cv_agg's auc: 0.74668 + 0.00404565
[1187]	cv_agg's auc: 0.746695 + 0.00402637
[1188]	cv_agg's auc: 0.746697 + 0.00401835
[1189]	cv_agg's auc: 0.746689 + 0.00401698
[1190]	cv_agg's auc: 0.746686 + 0.00403382
[1191]	cv_agg's auc: 0.746681 + 0.00403097
[1192]	cv_agg's auc: 0.746673 + 0.00404268
[1193]	cv_agg's auc: 0.746688 + 0.0040479
[1194]	cv_agg's auc: 0.746698 + 0.00403052
[1195]	cv_agg's auc: 0.746715 + 0.00402629
[1196]	cv_agg's auc: 0.746696 + 0.00400678
[1197]	cv_agg's auc: 0.746688 + 0.00400761
[1198]	cv_agg's auc: 0.746687 + 0.00400341
[1199]	cv_agg's auc: 0.746705 + 0.00398103
[1200]	cv_agg's auc: 0.746707 + 0.00396945
[1201]	cv_agg's auc: 0.74672 + 0.00399732
[1202]	cv_agg's auc: 0.746717 + 0.00400527
[1203]	cv_agg's auc: 0.746725 + 0.00400979
[1204]	cv_agg's

[1373]	cv_agg's auc: 0.746742 + 0.00420127
[1374]	cv_agg's auc: 0.746742 + 0.00420283
[1375]	cv_agg's auc: 0.746738 + 0.00421391
[1376]	cv_agg's auc: 0.74672 + 0.00422061
[1377]	cv_agg's auc: 0.746719 + 0.00422439
[1378]	cv_agg's auc: 0.746719 + 0.0042432
[1379]	cv_agg's auc: 0.746728 + 0.00423112
[1380]	cv_agg's auc: 0.746725 + 0.00422382
[1381]	cv_agg's auc: 0.746719 + 0.00421635
[1382]	cv_agg's auc: 0.74671 + 0.00420909
[1383]	cv_agg's auc: 0.746712 + 0.00421477
[1384]	cv_agg's auc: 0.746727 + 0.00422826
[1385]	cv_agg's auc: 0.746717 + 0.00420911
[1386]	cv_agg's auc: 0.746728 + 0.00422529
[1387]	cv_agg's auc: 0.746735 + 0.00421026
[1388]	cv_agg's auc: 0.746752 + 0.00420618
[1389]	cv_agg's auc: 0.746751 + 0.00419868
[1390]	cv_agg's auc: 0.746777 + 0.00422134
[1391]	cv_agg's auc: 0.74676 + 0.00423615
[1392]	cv_agg's auc: 0.746761 + 0.0042122
[1393]	cv_agg's auc: 0.746766 + 0.00419971
[1394]	cv_agg's auc: 0.746769 + 0.00419488
[1395]	cv_agg's auc: 0.746753 + 0.00423345
[1396]	cv_agg's 

In [83]:
check_train_score(params_db, lgb_train_db, new_train[features_db], new_train.target, cv, 1348)

array([0.90433487, 0.74686359])

# 3. Feature selection

### 3.1 Statistical approach

In [67]:
from sklearn.feature_selection import VarianceThreshold

thresholds = np.arange(.05, 1.0, .05)

for thres in thresholds:
    print('{:.2f}:{}'.format(thres, VarianceThreshold(thres).fit_transform(data[n_features]).shape))

0.05:(34666, 92)
0.10:(34666, 60)
0.15:(34666, 37)
0.20:(34666, 23)
0.25:(34666, 11)
0.30:(34666, 10)
0.35:(34666, 10)
0.40:(34666, 10)
0.45:(34666, 10)
0.50:(34666, 10)
0.55:(34666, 10)
0.60:(34666, 10)
0.65:(34666, 10)
0.70:(34666, 10)
0.75:(34666, 10)
0.80:(34666, 10)
0.85:(34666, 10)
0.90:(34666, 10)
0.95:(34666, 10)


### 3.2 Meta-classifier

In [84]:
#using results of RandomForestClassifier in the first part of notebook
params_meta = {
    'objective':'binary',
    'seed': 42,
    
    'learning_rate': 0.009,
    'num_threads': 3,
    'metric': 'auc',
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 1e-3,
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq': 1,
    'num_leaves': 31
}


lgb_train_meta = lgb.Dataset(new_train[non_red_features], label=new_train.target, free_raw_data=False)

In [85]:
%%time
result = lgb.cv(params_meta, lgb_train_meta, n_rounds, folds=cv.split(new_train[non_red_features], new_train.target), early_stopping_rounds=100, verbose_eval=1)

[1]	cv_agg's auc: 0.687726 + 0.00661299
[2]	cv_agg's auc: 0.697262 + 0.0062056
[3]	cv_agg's auc: 0.701759 + 0.00716894
[4]	cv_agg's auc: 0.704945 + 0.00787498
[5]	cv_agg's auc: 0.706869 + 0.00788746
[6]	cv_agg's auc: 0.707131 + 0.00818404
[7]	cv_agg's auc: 0.708498 + 0.00755109
[8]	cv_agg's auc: 0.708928 + 0.0079016
[9]	cv_agg's auc: 0.710131 + 0.00786315
[10]	cv_agg's auc: 0.710886 + 0.00672785
[11]	cv_agg's auc: 0.71278 + 0.00703191
[12]	cv_agg's auc: 0.713007 + 0.0065113
[13]	cv_agg's auc: 0.71308 + 0.00724877
[14]	cv_agg's auc: 0.713289 + 0.00745411
[15]	cv_agg's auc: 0.713026 + 0.00751187
[16]	cv_agg's auc: 0.71294 + 0.00724367
[17]	cv_agg's auc: 0.713318 + 0.00763359
[18]	cv_agg's auc: 0.713198 + 0.00728583
[19]	cv_agg's auc: 0.713066 + 0.00712746
[20]	cv_agg's auc: 0.713314 + 0.00706904
[21]	cv_agg's auc: 0.713487 + 0.0072125
[22]	cv_agg's auc: 0.713612 + 0.00727064
[23]	cv_agg's auc: 0.713765 + 0.00735069
[24]	cv_agg's auc: 0.713686 + 0.00748295
[25]	cv_agg's auc: 0.713965 + 0.

[200]	cv_agg's auc: 0.729512 + 0.00551757
[201]	cv_agg's auc: 0.72956 + 0.00551336
[202]	cv_agg's auc: 0.729655 + 0.00547784
[203]	cv_agg's auc: 0.729749 + 0.00545382
[204]	cv_agg's auc: 0.7298 + 0.00546881
[205]	cv_agg's auc: 0.729877 + 0.00545578
[206]	cv_agg's auc: 0.72998 + 0.00542761
[207]	cv_agg's auc: 0.730065 + 0.00540825
[208]	cv_agg's auc: 0.730132 + 0.00539186
[209]	cv_agg's auc: 0.730203 + 0.00539202
[210]	cv_agg's auc: 0.7303 + 0.00540125
[211]	cv_agg's auc: 0.73038 + 0.00541925
[212]	cv_agg's auc: 0.730479 + 0.0054216
[213]	cv_agg's auc: 0.730558 + 0.00542446
[214]	cv_agg's auc: 0.730615 + 0.00543196
[215]	cv_agg's auc: 0.730702 + 0.00539866
[216]	cv_agg's auc: 0.730811 + 0.00537819
[217]	cv_agg's auc: 0.730883 + 0.00539339
[218]	cv_agg's auc: 0.730959 + 0.00540589
[219]	cv_agg's auc: 0.731043 + 0.00541418
[220]	cv_agg's auc: 0.731128 + 0.00539183
[221]	cv_agg's auc: 0.731267 + 0.00542322
[222]	cv_agg's auc: 0.731342 + 0.00542161
[223]	cv_agg's auc: 0.731392 + 0.00539194


[397]	cv_agg's auc: 0.739749 + 0.00427248
[398]	cv_agg's auc: 0.739779 + 0.004254
[399]	cv_agg's auc: 0.739828 + 0.00423316
[400]	cv_agg's auc: 0.739897 + 0.00425287
[401]	cv_agg's auc: 0.739916 + 0.00422773
[402]	cv_agg's auc: 0.739935 + 0.00417859
[403]	cv_agg's auc: 0.739971 + 0.00417577
[404]	cv_agg's auc: 0.740026 + 0.00413136
[405]	cv_agg's auc: 0.740089 + 0.0041109
[406]	cv_agg's auc: 0.7401 + 0.00408353
[407]	cv_agg's auc: 0.740116 + 0.00405004
[408]	cv_agg's auc: 0.740156 + 0.00403812
[409]	cv_agg's auc: 0.740129 + 0.00401437
[410]	cv_agg's auc: 0.74015 + 0.00399041
[411]	cv_agg's auc: 0.740203 + 0.00400684
[412]	cv_agg's auc: 0.740211 + 0.00400231
[413]	cv_agg's auc: 0.740203 + 0.0039661
[414]	cv_agg's auc: 0.740211 + 0.00395671
[415]	cv_agg's auc: 0.740224 + 0.00393015
[416]	cv_agg's auc: 0.740288 + 0.0039152
[417]	cv_agg's auc: 0.740316 + 0.00392499
[418]	cv_agg's auc: 0.740347 + 0.0039217
[419]	cv_agg's auc: 0.740381 + 0.0038983
[420]	cv_agg's auc: 0.740397 + 0.00385322
[4

[594]	cv_agg's auc: 0.743534 + 0.00343006
[595]	cv_agg's auc: 0.74354 + 0.00344702
[596]	cv_agg's auc: 0.743564 + 0.00344102
[597]	cv_agg's auc: 0.743574 + 0.00344036
[598]	cv_agg's auc: 0.743587 + 0.00345707
[599]	cv_agg's auc: 0.74362 + 0.00346995
[600]	cv_agg's auc: 0.743649 + 0.00344822
[601]	cv_agg's auc: 0.743653 + 0.00344583
[602]	cv_agg's auc: 0.743651 + 0.00346
[603]	cv_agg's auc: 0.743651 + 0.00344303
[604]	cv_agg's auc: 0.743664 + 0.00343218
[605]	cv_agg's auc: 0.743675 + 0.00342048
[606]	cv_agg's auc: 0.743684 + 0.0034487
[607]	cv_agg's auc: 0.743701 + 0.00345053
[608]	cv_agg's auc: 0.743742 + 0.00342068
[609]	cv_agg's auc: 0.743778 + 0.00344273
[610]	cv_agg's auc: 0.743789 + 0.00344925
[611]	cv_agg's auc: 0.743786 + 0.00345999
[612]	cv_agg's auc: 0.743767 + 0.0034553
[613]	cv_agg's auc: 0.743785 + 0.00345304
[614]	cv_agg's auc: 0.743805 + 0.00342726
[615]	cv_agg's auc: 0.743824 + 0.0034568
[616]	cv_agg's auc: 0.743828 + 0.0034615
[617]	cv_agg's auc: 0.743836 + 0.00349582
[

[792]	cv_agg's auc: 0.745515 + 0.00321583
[793]	cv_agg's auc: 0.745504 + 0.00323717
[794]	cv_agg's auc: 0.745522 + 0.00323218
[795]	cv_agg's auc: 0.745531 + 0.00321465
[796]	cv_agg's auc: 0.745523 + 0.00322395
[797]	cv_agg's auc: 0.745495 + 0.00323431
[798]	cv_agg's auc: 0.745491 + 0.00323136
[799]	cv_agg's auc: 0.745486 + 0.00320146
[800]	cv_agg's auc: 0.745492 + 0.0032004
[801]	cv_agg's auc: 0.745497 + 0.0032232
[802]	cv_agg's auc: 0.745485 + 0.00322496
[803]	cv_agg's auc: 0.745466 + 0.00322
[804]	cv_agg's auc: 0.745466 + 0.00324974
[805]	cv_agg's auc: 0.74549 + 0.0032626
[806]	cv_agg's auc: 0.745518 + 0.00324252
[807]	cv_agg's auc: 0.745523 + 0.00324358
[808]	cv_agg's auc: 0.745516 + 0.00323128
[809]	cv_agg's auc: 0.7455 + 0.00322127
[810]	cv_agg's auc: 0.745529 + 0.00321409
[811]	cv_agg's auc: 0.745549 + 0.00321789
[812]	cv_agg's auc: 0.745561 + 0.00322728
[813]	cv_agg's auc: 0.745559 + 0.00320811
[814]	cv_agg's auc: 0.7456 + 0.00324206
[815]	cv_agg's auc: 0.745613 + 0.00323452
[81

[990]	cv_agg's auc: 0.746251 + 0.00326162
[991]	cv_agg's auc: 0.746247 + 0.0032652
[992]	cv_agg's auc: 0.746243 + 0.00324729
[993]	cv_agg's auc: 0.746251 + 0.00327024
[994]	cv_agg's auc: 0.746236 + 0.00328347
[995]	cv_agg's auc: 0.74625 + 0.00326364
[996]	cv_agg's auc: 0.746271 + 0.00327081
[997]	cv_agg's auc: 0.746273 + 0.00327238
[998]	cv_agg's auc: 0.74628 + 0.0032741
[999]	cv_agg's auc: 0.746286 + 0.00326707
[1000]	cv_agg's auc: 0.746284 + 0.00328144
[1001]	cv_agg's auc: 0.746276 + 0.00328596
[1002]	cv_agg's auc: 0.746289 + 0.00327265
[1003]	cv_agg's auc: 0.746292 + 0.003289
[1004]	cv_agg's auc: 0.746273 + 0.00329908
[1005]	cv_agg's auc: 0.746276 + 0.00331272
[1006]	cv_agg's auc: 0.746288 + 0.00332011
[1007]	cv_agg's auc: 0.746293 + 0.00331344
[1008]	cv_agg's auc: 0.746293 + 0.00334632
[1009]	cv_agg's auc: 0.746298 + 0.00334771
[1010]	cv_agg's auc: 0.746289 + 0.00333421
[1011]	cv_agg's auc: 0.746297 + 0.00336024
[1012]	cv_agg's auc: 0.746304 + 0.00337003
[1013]	cv_agg's auc: 0.7463

[1182]	cv_agg's auc: 0.746948 + 0.00323317
[1183]	cv_agg's auc: 0.746942 + 0.00322472
[1184]	cv_agg's auc: 0.746964 + 0.00321432
[1185]	cv_agg's auc: 0.746977 + 0.0032082
[1186]	cv_agg's auc: 0.746977 + 0.00320163
[1187]	cv_agg's auc: 0.74698 + 0.00319614
[1188]	cv_agg's auc: 0.746995 + 0.00318338
[1189]	cv_agg's auc: 0.747009 + 0.00318206
[1190]	cv_agg's auc: 0.746996 + 0.00318291
[1191]	cv_agg's auc: 0.746984 + 0.00316001
[1192]	cv_agg's auc: 0.746991 + 0.00319073
[1193]	cv_agg's auc: 0.747009 + 0.00317305
[1194]	cv_agg's auc: 0.747006 + 0.00316283
[1195]	cv_agg's auc: 0.747031 + 0.00315636
[1196]	cv_agg's auc: 0.747041 + 0.00315371
[1197]	cv_agg's auc: 0.747057 + 0.00314247
[1198]	cv_agg's auc: 0.747048 + 0.00315128
[1199]	cv_agg's auc: 0.747043 + 0.0031456
[1200]	cv_agg's auc: 0.747037 + 0.00313732
[1201]	cv_agg's auc: 0.747053 + 0.00314299
[1202]	cv_agg's auc: 0.747046 + 0.00313987
[1203]	cv_agg's auc: 0.747042 + 0.00313967
[1204]	cv_agg's auc: 0.747049 + 0.00312871
[1205]	cv_agg'

[1375]	cv_agg's auc: 0.747307 + 0.00335593
[1376]	cv_agg's auc: 0.747318 + 0.0033458
[1377]	cv_agg's auc: 0.747307 + 0.00335122
[1378]	cv_agg's auc: 0.747325 + 0.00335046
[1379]	cv_agg's auc: 0.747323 + 0.00333942
[1380]	cv_agg's auc: 0.74733 + 0.00333329
[1381]	cv_agg's auc: 0.747319 + 0.00334391
[1382]	cv_agg's auc: 0.747321 + 0.00335392
[1383]	cv_agg's auc: 0.747311 + 0.00336012
[1384]	cv_agg's auc: 0.747319 + 0.00337507
[1385]	cv_agg's auc: 0.747287 + 0.00336187
[1386]	cv_agg's auc: 0.747305 + 0.00337057
[1387]	cv_agg's auc: 0.747297 + 0.00333632
[1388]	cv_agg's auc: 0.747289 + 0.00333279
[1389]	cv_agg's auc: 0.747303 + 0.0033213
[1390]	cv_agg's auc: 0.747308 + 0.00333371
[1391]	cv_agg's auc: 0.747298 + 0.00331687
[1392]	cv_agg's auc: 0.747293 + 0.00330319
[1393]	cv_agg's auc: 0.747298 + 0.00331401
[1394]	cv_agg's auc: 0.747298 + 0.00332301
[1395]	cv_agg's auc: 0.747304 + 0.00333512
[1396]	cv_agg's auc: 0.747298 + 0.0033252
[1397]	cv_agg's auc: 0.747284 + 0.00331537
[1398]	cv_agg's

In [86]:
check_train_score(params_meta, lgb_train_meta, new_train[non_red_features], new_train.target, cv, 1373)

array([0.9057508 , 0.74733925])

### 3.3 Recursive Feature Selection

In [99]:
from mlxtend.feature_selection import SequentialFeatureSelector

selector = SequentialFeatureSelector(
    RandomForestClassifier(random_state=42), 
    scoring='roc_auc', 
    verbose=1, 
    k_features=330, 
    forward=False, 
    n_jobs=3,
    cv=cv
)
X_train, y_train = new_train[n_features].values, new_train.target.values
selector.fit(X_train, y_train)

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.3min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 17.2min
[Parallel(n_jobs=3)]: Done 358 out of 358 | elapsed: 135.2min finished
Features: 357/330[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.7min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 18.2min
[Parallel(n_jobs=3)]: Done 357 out of 357 | elapsed: 33.1min finished
Features: 356/330[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  3.9min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 17.2min

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

SequentialFeatureSelector(clone_estimator=True,
             cv=StratifiedKFold(n_splits=8, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
             floating=False, forward=False, k_features=330, n_jobs=3,
             pre_dispatch='2*n_jobs', scoring='roc_auc', verbose=1)

# 4. Target Encoding

In [90]:
from category_encoders.target_encoder import TargetEncoder

In [91]:
catencoder = TargetEncoder(cols=categorical+ordinal).fit(new_train[n_features], new_train.target)

In [92]:
enc_data = catencoder.transform(data[n_features])
enc_data['target'] = data.target
enc_train = enc_data[~enc_data.target.isna()]
enc_test = enc_data[enc_data.target.isna()]

In [93]:
params_tenc = {
    'objective':'binary',
    'seed': 42,
    
    'learning_rate': 0.009,
    'num_threads': 3,
    'metric': 'auc',
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 1e-3,
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq': 1,
    'num_leaves': 31
}


lgb_train_tenc = lgb.Dataset(enc_train[n_features], label=enc_train.target, free_raw_data=False)

In [94]:
%%time
result = lgb.cv(params_tenc, lgb_train_tenc, n_rounds, folds=cv.split(enc_train[n_features], enc_train.target), early_stopping_rounds=100, verbose_eval=1)

[1]	cv_agg's auc: 0.674809 + 0.0139553
[2]	cv_agg's auc: 0.694832 + 0.0068866
[3]	cv_agg's auc: 0.702477 + 0.00484935
[4]	cv_agg's auc: 0.704832 + 0.00585901
[5]	cv_agg's auc: 0.707603 + 0.00709883
[6]	cv_agg's auc: 0.709274 + 0.00610906
[7]	cv_agg's auc: 0.709942 + 0.00543303
[8]	cv_agg's auc: 0.710913 + 0.00550042
[9]	cv_agg's auc: 0.712211 + 0.00505968
[10]	cv_agg's auc: 0.712493 + 0.00492951
[11]	cv_agg's auc: 0.712476 + 0.00495594
[12]	cv_agg's auc: 0.712382 + 0.00494034
[13]	cv_agg's auc: 0.712384 + 0.00504091
[14]	cv_agg's auc: 0.712856 + 0.00514803
[15]	cv_agg's auc: 0.713476 + 0.00540101
[16]	cv_agg's auc: 0.71349 + 0.00541102
[17]	cv_agg's auc: 0.713556 + 0.00551373
[18]	cv_agg's auc: 0.713794 + 0.00596819
[19]	cv_agg's auc: 0.714031 + 0.0059994
[20]	cv_agg's auc: 0.714276 + 0.00586238
[21]	cv_agg's auc: 0.714782 + 0.0060432
[22]	cv_agg's auc: 0.7151 + 0.00604983
[23]	cv_agg's auc: 0.715189 + 0.00624501
[24]	cv_agg's auc: 0.715023 + 0.00642912
[25]	cv_agg's auc: 0.714908 + 0.

[201]	cv_agg's auc: 0.730401 + 0.00531456
[202]	cv_agg's auc: 0.730475 + 0.00534306
[203]	cv_agg's auc: 0.730544 + 0.00536357
[204]	cv_agg's auc: 0.73059 + 0.00537098
[205]	cv_agg's auc: 0.730618 + 0.005368
[206]	cv_agg's auc: 0.730701 + 0.00537723
[207]	cv_agg's auc: 0.730879 + 0.00530097
[208]	cv_agg's auc: 0.730966 + 0.00522855
[209]	cv_agg's auc: 0.730988 + 0.00519637
[210]	cv_agg's auc: 0.731023 + 0.0052138
[211]	cv_agg's auc: 0.731072 + 0.00517361
[212]	cv_agg's auc: 0.731114 + 0.00519857
[213]	cv_agg's auc: 0.731183 + 0.0052073
[214]	cv_agg's auc: 0.731257 + 0.00518433
[215]	cv_agg's auc: 0.731391 + 0.00518453
[216]	cv_agg's auc: 0.731475 + 0.00524683
[217]	cv_agg's auc: 0.731524 + 0.0052233
[218]	cv_agg's auc: 0.731549 + 0.00522759
[219]	cv_agg's auc: 0.731685 + 0.00523691
[220]	cv_agg's auc: 0.731715 + 0.00524723
[221]	cv_agg's auc: 0.731802 + 0.00523747
[222]	cv_agg's auc: 0.731893 + 0.00519181
[223]	cv_agg's auc: 0.732011 + 0.00523059
[224]	cv_agg's auc: 0.732129 + 0.0052355

[397]	cv_agg's auc: 0.740122 + 0.00447689
[398]	cv_agg's auc: 0.740127 + 0.00445634
[399]	cv_agg's auc: 0.740175 + 0.00445522
[400]	cv_agg's auc: 0.740182 + 0.00444556
[401]	cv_agg's auc: 0.740249 + 0.00445768
[402]	cv_agg's auc: 0.740263 + 0.00444647
[403]	cv_agg's auc: 0.740299 + 0.00448007
[404]	cv_agg's auc: 0.740333 + 0.00447375
[405]	cv_agg's auc: 0.74037 + 0.00449386
[406]	cv_agg's auc: 0.740411 + 0.00448699
[407]	cv_agg's auc: 0.740429 + 0.00448604
[408]	cv_agg's auc: 0.740468 + 0.00445652
[409]	cv_agg's auc: 0.740486 + 0.00445179
[410]	cv_agg's auc: 0.740492 + 0.00446051
[411]	cv_agg's auc: 0.740509 + 0.00445045
[412]	cv_agg's auc: 0.740553 + 0.00446855
[413]	cv_agg's auc: 0.740585 + 0.00446253
[414]	cv_agg's auc: 0.74059 + 0.0044449
[415]	cv_agg's auc: 0.740621 + 0.00447301
[416]	cv_agg's auc: 0.740689 + 0.0044736
[417]	cv_agg's auc: 0.740724 + 0.0044662
[418]	cv_agg's auc: 0.740756 + 0.0044684
[419]	cv_agg's auc: 0.740795 + 0.00446241
[420]	cv_agg's auc: 0.740818 + 0.0044373

[595]	cv_agg's auc: 0.74402 + 0.00414977
[596]	cv_agg's auc: 0.744043 + 0.00413812
[597]	cv_agg's auc: 0.744062 + 0.00413698
[598]	cv_agg's auc: 0.744049 + 0.00413736
[599]	cv_agg's auc: 0.744064 + 0.00413866
[600]	cv_agg's auc: 0.744048 + 0.00416066
[601]	cv_agg's auc: 0.744077 + 0.00413758
[602]	cv_agg's auc: 0.744099 + 0.00415657
[603]	cv_agg's auc: 0.74409 + 0.00416376
[604]	cv_agg's auc: 0.74411 + 0.00416694
[605]	cv_agg's auc: 0.744133 + 0.0041557
[606]	cv_agg's auc: 0.744157 + 0.00414676
[607]	cv_agg's auc: 0.744168 + 0.0041554
[608]	cv_agg's auc: 0.744154 + 0.00412799
[609]	cv_agg's auc: 0.744181 + 0.00413891
[610]	cv_agg's auc: 0.744165 + 0.00414625
[611]	cv_agg's auc: 0.744172 + 0.00414614
[612]	cv_agg's auc: 0.744162 + 0.00416792
[613]	cv_agg's auc: 0.74417 + 0.00416747
[614]	cv_agg's auc: 0.74418 + 0.0041459
[615]	cv_agg's auc: 0.744184 + 0.00412442
[616]	cv_agg's auc: 0.744202 + 0.00412083
[617]	cv_agg's auc: 0.744158 + 0.00413358
[618]	cv_agg's auc: 0.74419 + 0.004119
[61

[794]	cv_agg's auc: 0.745813 + 0.00412386
[795]	cv_agg's auc: 0.745828 + 0.00411699
[796]	cv_agg's auc: 0.745822 + 0.00409623
[797]	cv_agg's auc: 0.745802 + 0.00411099
[798]	cv_agg's auc: 0.745811 + 0.00407293
[799]	cv_agg's auc: 0.745831 + 0.00405406
[800]	cv_agg's auc: 0.745839 + 0.0040645
[801]	cv_agg's auc: 0.745861 + 0.0040839
[802]	cv_agg's auc: 0.745857 + 0.00410712
[803]	cv_agg's auc: 0.745846 + 0.00410289
[804]	cv_agg's auc: 0.745846 + 0.00408307
[805]	cv_agg's auc: 0.745847 + 0.00406632
[806]	cv_agg's auc: 0.745855 + 0.00405374
[807]	cv_agg's auc: 0.74585 + 0.00406538
[808]	cv_agg's auc: 0.745839 + 0.00407198
[809]	cv_agg's auc: 0.745817 + 0.00406677
[810]	cv_agg's auc: 0.745835 + 0.00405573
[811]	cv_agg's auc: 0.745829 + 0.00406788
[812]	cv_agg's auc: 0.745842 + 0.0040731
[813]	cv_agg's auc: 0.745863 + 0.00408379
[814]	cv_agg's auc: 0.745869 + 0.00406699
[815]	cv_agg's auc: 0.745873 + 0.00405292
[816]	cv_agg's auc: 0.74587 + 0.00404833
[817]	cv_agg's auc: 0.745885 + 0.004033

[991]	cv_agg's auc: 0.746604 + 0.00404958
[992]	cv_agg's auc: 0.746594 + 0.00407099
[993]	cv_agg's auc: 0.746602 + 0.00408495
[994]	cv_agg's auc: 0.746594 + 0.00408252
[995]	cv_agg's auc: 0.746589 + 0.00406949
[996]	cv_agg's auc: 0.746584 + 0.00409072
[997]	cv_agg's auc: 0.746581 + 0.00410109
[998]	cv_agg's auc: 0.746569 + 0.00411414
[999]	cv_agg's auc: 0.746555 + 0.00412935
[1000]	cv_agg's auc: 0.746548 + 0.0041094
[1001]	cv_agg's auc: 0.746552 + 0.0041153
[1002]	cv_agg's auc: 0.746539 + 0.00411302
[1003]	cv_agg's auc: 0.746534 + 0.00410997
[1004]	cv_agg's auc: 0.746563 + 0.00413364
[1005]	cv_agg's auc: 0.746552 + 0.0041295
[1006]	cv_agg's auc: 0.746559 + 0.00411205
[1007]	cv_agg's auc: 0.746552 + 0.00411372
[1008]	cv_agg's auc: 0.746544 + 0.00411118
[1009]	cv_agg's auc: 0.74654 + 0.00411073
[1010]	cv_agg's auc: 0.74655 + 0.00409467
[1011]	cv_agg's auc: 0.746573 + 0.00411457
[1012]	cv_agg's auc: 0.746581 + 0.00410662
[1013]	cv_agg's auc: 0.746584 + 0.00410128
[1014]	cv_agg's auc: 0.74

[1185]	cv_agg's auc: 0.746945 + 0.00405013
[1186]	cv_agg's auc: 0.746955 + 0.00404418
[1187]	cv_agg's auc: 0.746961 + 0.0040483
[1188]	cv_agg's auc: 0.746984 + 0.00404988
[1189]	cv_agg's auc: 0.747004 + 0.00402506
[1190]	cv_agg's auc: 0.747006 + 0.0040131
[1191]	cv_agg's auc: 0.747009 + 0.00404209
[1192]	cv_agg's auc: 0.747019 + 0.00404141
[1193]	cv_agg's auc: 0.747026 + 0.00402755
[1194]	cv_agg's auc: 0.747024 + 0.00403086
[1195]	cv_agg's auc: 0.747032 + 0.00404355
[1196]	cv_agg's auc: 0.747053 + 0.00405183
[1197]	cv_agg's auc: 0.747059 + 0.00405223
[1198]	cv_agg's auc: 0.747067 + 0.00403607
[1199]	cv_agg's auc: 0.747077 + 0.004024
[1200]	cv_agg's auc: 0.747057 + 0.00401344
[1201]	cv_agg's auc: 0.747078 + 0.00403401
[1202]	cv_agg's auc: 0.747077 + 0.00403243
[1203]	cv_agg's auc: 0.747088 + 0.00404455
[1204]	cv_agg's auc: 0.74709 + 0.00405207
[1205]	cv_agg's auc: 0.7471 + 0.00407588
[1206]	cv_agg's auc: 0.747098 + 0.00406459
[1207]	cv_agg's auc: 0.747116 + 0.00405779
[1208]	cv_agg's au

[1379]	cv_agg's auc: 0.747448 + 0.00423655
[1380]	cv_agg's auc: 0.747448 + 0.00423111
[1381]	cv_agg's auc: 0.747435 + 0.00422766
[1382]	cv_agg's auc: 0.747431 + 0.00421224
[1383]	cv_agg's auc: 0.747429 + 0.00419099
[1384]	cv_agg's auc: 0.747443 + 0.00420931
[1385]	cv_agg's auc: 0.747436 + 0.00419533
[1386]	cv_agg's auc: 0.747436 + 0.00419392
[1387]	cv_agg's auc: 0.747446 + 0.00417779
[1388]	cv_agg's auc: 0.747426 + 0.00417095
[1389]	cv_agg's auc: 0.74742 + 0.00417425
[1390]	cv_agg's auc: 0.747429 + 0.00418488
[1391]	cv_agg's auc: 0.74742 + 0.00417415
[1392]	cv_agg's auc: 0.747421 + 0.00417876
[1393]	cv_agg's auc: 0.747428 + 0.00418945
[1394]	cv_agg's auc: 0.747435 + 0.00419586
[1395]	cv_agg's auc: 0.747422 + 0.0042078
[1396]	cv_agg's auc: 0.747415 + 0.00420564
[1397]	cv_agg's auc: 0.747414 + 0.00420054
[1398]	cv_agg's auc: 0.747418 + 0.00420648
[1399]	cv_agg's auc: 0.747423 + 0.00422213
[1400]	cv_agg's auc: 0.747437 + 0.00421032
[1401]	cv_agg's auc: 0.747445 + 0.00420979
[1402]	cv_agg'

In [95]:
check_train_score(params_tenc, lgb_train_tenc, enc_train[n_features], enc_train.target, cv, 1428)

array([0.90972477, 0.74752022])

# 5. Xgbfir

In [97]:
import xgboost as xgb
import xgbfir

params_xgb = {
    'objective': 'binary:logistic',
    'eta': 0.009,
    'silent': 1,
    'nthread': 3,
    'seed': 19,
    'eval_metric': 'auc',
    
    #new parameters
    'subsample': 0.75,
    'colsample_bytree': 0.66,
    'tree_method': 'hist',
    'grow_policy': 'lossguide',
    'max_depth': 7,
    'max_leaves': 64
}

xgb_train = xgb.DMatrix(new_train[n_features], new_train.target, feature_names=n_features)

In [98]:
results = xgb.cv(
    params_xgb, 
    xgb_train, 
    num_boost_round=2000,
    early_stopping_rounds=100,
    folds=cv, 
    verbose_eval=1
)

[0]	train-auc:0.707705+0.00157167	test-auc:0.678135+0.00175024
[1]	train-auc:0.731478+0.00238443	test-auc:0.693883+0.00464126
[2]	train-auc:0.738591+0.000595119	test-auc:0.696871+0.00454575
[3]	train-auc:0.744639+0.00265329	test-auc:0.70108+0.00635427
[4]	train-auc:0.74936+0.00347451	test-auc:0.703789+0.00563102
[5]	train-auc:0.752672+0.00370203	test-auc:0.706233+0.00564373
[6]	train-auc:0.755489+0.0029359	test-auc:0.708307+0.00471755
[7]	train-auc:0.75656+0.00290069	test-auc:0.709191+0.00505493
[8]	train-auc:0.758183+0.00343842	test-auc:0.710269+0.00550296
[9]	train-auc:0.759324+0.00308307	test-auc:0.711604+0.00500045
[10]	train-auc:0.760694+0.00291796	test-auc:0.712544+0.00412354
[11]	train-auc:0.760848+0.00327291	test-auc:0.712947+0.00431924
[12]	train-auc:0.761376+0.00307521	test-auc:0.712863+0.00427588
[13]	train-auc:0.76153+0.00310042	test-auc:0.713242+0.00423776
[14]	train-auc:0.7623+0.00259271	test-auc:0.713785+0.00390701
[15]	train-auc:0.762289+0.00246836	test-auc:0.713923+0.0

[129]	train-auc:0.796579+0.0007721	test-auc:0.725611+0.00412124
[130]	train-auc:0.796708+0.000754472	test-auc:0.725622+0.00410531
[131]	train-auc:0.796986+0.000758372	test-auc:0.725587+0.00406054
[132]	train-auc:0.797316+0.000719502	test-auc:0.725625+0.00411704
[133]	train-auc:0.79752+0.000702143	test-auc:0.725613+0.00409004
[134]	train-auc:0.797787+0.00070253	test-auc:0.725677+0.0040874
[135]	train-auc:0.798051+0.000841628	test-auc:0.725787+0.00406157
[136]	train-auc:0.798332+0.000919698	test-auc:0.725936+0.00415349
[137]	train-auc:0.79866+0.000892218	test-auc:0.725961+0.00418907
[138]	train-auc:0.799078+0.000949579	test-auc:0.726005+0.00419259
[139]	train-auc:0.799333+0.000918732	test-auc:0.72605+0.0041213
[140]	train-auc:0.799816+0.000903687	test-auc:0.726182+0.00412406
[141]	train-auc:0.800069+0.00095936	test-auc:0.7262+0.00409089
[142]	train-auc:0.800271+0.000952053	test-auc:0.72631+0.00410248
[143]	train-auc:0.800598+0.000932906	test-auc:0.726385+0.00411237
[144]	train-auc:0.8009

[256]	train-auc:0.831887+0.00143591	test-auc:0.733341+0.00256699
[257]	train-auc:0.832148+0.00144184	test-auc:0.733365+0.00258398
[258]	train-auc:0.832311+0.00142036	test-auc:0.733414+0.00260967
[259]	train-auc:0.832559+0.0013578	test-auc:0.733512+0.00258043
[260]	train-auc:0.832787+0.00134327	test-auc:0.733512+0.00258861
[261]	train-auc:0.833043+0.00133573	test-auc:0.733579+0.00259039
[262]	train-auc:0.833321+0.00131736	test-auc:0.733657+0.00251305
[263]	train-auc:0.833484+0.00136376	test-auc:0.7337+0.00254971
[264]	train-auc:0.833758+0.00137737	test-auc:0.733741+0.00255929
[265]	train-auc:0.833973+0.00143785	test-auc:0.733772+0.0025683
[266]	train-auc:0.834267+0.00137769	test-auc:0.733843+0.00260613
[267]	train-auc:0.834517+0.00133342	test-auc:0.733864+0.00267738
[268]	train-auc:0.834792+0.00139638	test-auc:0.733917+0.00265644
[269]	train-auc:0.835019+0.00136539	test-auc:0.733939+0.00263037
[270]	train-auc:0.835301+0.00140339	test-auc:0.734035+0.00259273
[271]	train-auc:0.835529+0.00

[383]	train-auc:0.861405+0.00139165	test-auc:0.738004+0.00100754
[384]	train-auc:0.861634+0.001427	test-auc:0.738037+0.000971929
[385]	train-auc:0.86186+0.00138997	test-auc:0.738037+0.000975538
[386]	train-auc:0.862039+0.00146101	test-auc:0.738043+0.000964797
[387]	train-auc:0.862247+0.00146639	test-auc:0.738105+0.000948379
[388]	train-auc:0.862486+0.00145153	test-auc:0.73808+0.000978622
[389]	train-auc:0.862682+0.00146125	test-auc:0.738092+0.000997072
[390]	train-auc:0.862857+0.00148865	test-auc:0.738093+0.0010173
[391]	train-auc:0.863043+0.00144487	test-auc:0.738126+0.001022
[392]	train-auc:0.863194+0.00145539	test-auc:0.738144+0.00103938
[393]	train-auc:0.863392+0.00148948	test-auc:0.738155+0.00103347
[394]	train-auc:0.863551+0.00152307	test-auc:0.738194+0.00104103
[395]	train-auc:0.86381+0.00148449	test-auc:0.738214+0.00105026
[396]	train-auc:0.864024+0.00144961	test-auc:0.738265+0.00102078
[397]	train-auc:0.864252+0.0015151	test-auc:0.738335+0.000998284
[398]	train-auc:0.864391+0.

[509]	train-auc:0.883046+0.00170512	test-auc:0.740336+0.000277316
[510]	train-auc:0.883168+0.0016677	test-auc:0.740334+0.000259135
[511]	train-auc:0.883318+0.00161361	test-auc:0.740364+0.000254522
[512]	train-auc:0.883453+0.00162206	test-auc:0.740367+0.000282989
[513]	train-auc:0.88362+0.00166009	test-auc:0.740386+0.000286699
[514]	train-auc:0.883806+0.00167627	test-auc:0.74042+0.000268423
[515]	train-auc:0.883966+0.00164776	test-auc:0.740412+0.000220705
[516]	train-auc:0.884122+0.00163518	test-auc:0.740402+0.000230985
[517]	train-auc:0.884303+0.00162383	test-auc:0.74042+0.000250574
[518]	train-auc:0.884435+0.00161741	test-auc:0.740446+0.000252463
[519]	train-auc:0.884629+0.00157086	test-auc:0.740433+0.000259492
[520]	train-auc:0.884781+0.00157847	test-auc:0.740458+0.000259243
[521]	train-auc:0.884916+0.00161266	test-auc:0.740436+0.000263358
[522]	train-auc:0.885092+0.00159727	test-auc:0.740428+0.000265713
[523]	train-auc:0.885233+0.00161277	test-auc:0.740432+0.000297686
[524]	train-au

[634]	train-auc:0.900594+0.00141848	test-auc:0.741571+0.000588219
[635]	train-auc:0.900784+0.00141069	test-auc:0.741595+0.000572091
[636]	train-auc:0.90088+0.00143878	test-auc:0.741602+0.000580209
[637]	train-auc:0.901015+0.0014154	test-auc:0.741595+0.000583111
[638]	train-auc:0.901112+0.00141043	test-auc:0.74161+0.000584884
[639]	train-auc:0.901235+0.00142939	test-auc:0.741595+0.000553989
[640]	train-auc:0.901353+0.00142598	test-auc:0.741584+0.000581892
[641]	train-auc:0.901452+0.00146037	test-auc:0.741597+0.000574042
[642]	train-auc:0.901595+0.00147557	test-auc:0.741607+0.000573083
[643]	train-auc:0.901779+0.00145575	test-auc:0.741567+0.000583652
[644]	train-auc:0.901919+0.00145844	test-auc:0.741584+0.000611252
[645]	train-auc:0.902066+0.00143822	test-auc:0.741601+0.000612923
[646]	train-auc:0.902207+0.00140886	test-auc:0.741616+0.000623924
[647]	train-auc:0.902315+0.00143389	test-auc:0.74163+0.00063642
[648]	train-auc:0.90244+0.00141907	test-auc:0.74168+0.000646073
[649]	train-auc:0

[759]	train-auc:0.915899+0.00134055	test-auc:0.741938+0.000910246
[760]	train-auc:0.916062+0.00133651	test-auc:0.741973+0.000929168
[761]	train-auc:0.91618+0.00132156	test-auc:0.74198+0.000926019
[762]	train-auc:0.91626+0.00131144	test-auc:0.741985+0.000940894
[763]	train-auc:0.916376+0.00133926	test-auc:0.742002+0.000954053
[764]	train-auc:0.916525+0.00133498	test-auc:0.741976+0.000951842
[765]	train-auc:0.916671+0.0013102	test-auc:0.741949+0.000976862
[766]	train-auc:0.91674+0.00130963	test-auc:0.741926+0.000997812
[767]	train-auc:0.916839+0.00127988	test-auc:0.741922+0.00100127
[768]	train-auc:0.916993+0.0012771	test-auc:0.741925+0.000993798
[769]	train-auc:0.917054+0.00127381	test-auc:0.741942+0.000994268
[770]	train-auc:0.917159+0.00128108	test-auc:0.741952+0.00100161
[771]	train-auc:0.917237+0.00124404	test-auc:0.74197+0.00100808
[772]	train-auc:0.917374+0.00122478	test-auc:0.741955+0.00103637
[773]	train-auc:0.917503+0.00124824	test-auc:0.741946+0.00102575
[774]	train-auc:0.9176

[886]	train-auc:0.928494+0.00112854	test-auc:0.742183+0.00167368
[887]	train-auc:0.928599+0.00110553	test-auc:0.74221+0.00168624
[888]	train-auc:0.928708+0.0010698	test-auc:0.742204+0.00168071
[889]	train-auc:0.928803+0.00107426	test-auc:0.742202+0.00166918
[890]	train-auc:0.928885+0.00108322	test-auc:0.742186+0.0016868
[891]	train-auc:0.928966+0.00105192	test-auc:0.742193+0.00169862
[892]	train-auc:0.929048+0.00103497	test-auc:0.742184+0.00167447
[893]	train-auc:0.929172+0.00106145	test-auc:0.74219+0.00169202
[894]	train-auc:0.929257+0.0010444	test-auc:0.742196+0.00168951
[895]	train-auc:0.929351+0.0010613	test-auc:0.742192+0.00168043
[896]	train-auc:0.929468+0.00104149	test-auc:0.742166+0.00166367
[897]	train-auc:0.929537+0.00101577	test-auc:0.74217+0.0016622
[898]	train-auc:0.929597+0.00102203	test-auc:0.742163+0.00166418
[899]	train-auc:0.929672+0.000997419	test-auc:0.742173+0.00166427
[900]	train-auc:0.929778+0.000966178	test-auc:0.742188+0.0016908
[901]	train-auc:0.929856+0.00096

In [102]:
base_xgb = xgb.train(params_xgb, xgb_train, 887)

In [103]:
xgbfir.saveXgbFI(
    base_xgb, 
    feature_names=n_features, 
    OutputXlsxFile='xgb_imp_.xlsx'
)

In [104]:
interdepth0 = pd.read_excel('xgb_imp_.xlsx', sheet_name='Interaction Depth 0')
interdepth1 = pd.read_excel('xgb_imp_.xlsx', sheet_name='Interaction Depth 1')

In [105]:
interdepth1.sort_values(by='Expected Gain', ascending=False)

,Interaction,Gain,FScore,wFScore,Average wFScore,Average Gain,Expected Gain,Gain Rank,FScore Rank,wFScore Rank,Avg wFScore Rank,Avg Gain Rank,Expected Gain Rank,Average Rank,Average Tree Index,Average Tree Depth
0,338|339,7026.01480,27,15.952212,0.590823,260.222770,4164.203020,1,1,1,3,9,1,2.666667,42.296296,1.000000
1,329|338,6895.62270,22,8.994586,0.408845,313.437395,2806.415586,2,5,3,22,4,2,6.333333,40.500000,1.000000
2,144|338,3654.17620,14,8.261341,0.590096,261.012586,2164.162605,3,19,4,4,8,3,6.833333,42.071429,1.000000
6,295|86,2668.80470,12,7.117464,0.593122,222.400392,1758.303821,7,22,6,2,15,4,9.333333,47.083333,1.666667
7,233|338,2628.51040,12,7.050097,0.587508,219.042533,1549.334400,8,23,7,6,16,5,10.833333,57.500000,1.000000
3,338|91,3550.68810,12,4.937651,0.411471,295.890675,1449.357048,4,21,11,21,5,6,11.333333,49.416667,1.000000
10,337|86,2274.91650,9,4.681564,0.520174,252.768500,1179.639291,11,41,13,10,10,7,15.333333,54.555556,1.000000
13,338|34,1955.65470,8,4.713383,0.589173,244.456838,1156.307661,14,43,12,5,11,8,15.500000,48.500000,1.000000
8,338|4,2424.87030,10,4.135828,0.413583,242.487030,1001.704381,9,30,14,19,12,9,15.500000,62.100000,1.000000
9,328|338,2403.72750,9,3.710428,0.412270,267.080833,986.477254,10,40,17,20,6,10,17.166667,55.222222,1.000000


In [100]:
features_pairs = [('338', '339'), ('329', '338'), ('144', '338')]

In [109]:
def make_feature(f1, f2, df):
    temp_df = df.copy()
    if (f1 in binary) & (f2 in binary):
        nf = '{}or{}'.format(f1, f2)
        temp_df[nf] = np.where(
            temp_df[f1] == temp_df[f2],
            0,
            1
        )
    if ((f1 in binary) & (f2 not in binary)) | ((f1 not in binary) & (f2 in binary)):
        nf = '{}*{}'.format(f1, f2)
        temp_df[nf] = temp_df[f1]*temp_df[f2]
    return temp_df, nf

In [110]:
data_gen = data.copy()
neofeatures = []
for i, j in features_pairs:
    data_gen, nfg = make_feature(i, j, data_gen)
    neofeatures.append(nfg)

In [112]:
gen_features = n_features + neofeatures

In [113]:
new_train = data_gen[~data_gen.target.isna()]
new_test = data_gen[data_gen.target.isna()]

In [111]:
xgb_train_gen = xgb.DMatrix(new_train[gen_features], new_train.target, feature_names=gen_features)

results = xgb.cv(
    params_xgb, 
    xgb_train_gen, 
    num_boost_round=2000,
    early_stopping_rounds=100,
    folds=cv, 
    verbose_eval=1
)

[0]	train-auc:0.712315+0.00299182	test-auc:0.683179+0.00937549
[1]	train-auc:0.730024+0.00266744	test-auc:0.696445+0.00491661
[2]	train-auc:0.738728+0.00453503	test-auc:0.70082+0.00466836
[3]	train-auc:0.742342+0.00505	test-auc:0.704388+0.00493824
[4]	train-auc:0.746098+0.00385859	test-auc:0.706784+0.00545042
[5]	train-auc:0.749169+0.0019269	test-auc:0.707671+0.00582743
[6]	train-auc:0.75105+0.000743517	test-auc:0.708912+0.00539669
[7]	train-auc:0.752957+0.000691518	test-auc:0.709935+0.00461503
[8]	train-auc:0.755224+0.00153108	test-auc:0.712138+0.0057966
[9]	train-auc:0.756835+0.0025994	test-auc:0.712944+0.00535636
[10]	train-auc:0.758138+0.0022031	test-auc:0.714056+0.00506065
[11]	train-auc:0.759247+0.00230078	test-auc:0.714302+0.00458045
[12]	train-auc:0.759596+0.00123842	test-auc:0.714692+0.00433597
[13]	train-auc:0.76026+0.00106947	test-auc:0.715197+0.00427369
[14]	train-auc:0.760608+0.000636394	test-auc:0.715364+0.00438116
[15]	train-auc:0.761252+0.000689339	test-auc:0.716007+0.0

[128]	train-auc:0.796768+0.00048456	test-auc:0.72629+0.00351858
[129]	train-auc:0.797015+0.000450838	test-auc:0.726353+0.00339697
[130]	train-auc:0.797218+0.000508699	test-auc:0.72651+0.0034018
[131]	train-auc:0.797609+0.000456877	test-auc:0.726591+0.00337982
[132]	train-auc:0.797879+0.000434523	test-auc:0.726715+0.00333968
[133]	train-auc:0.79814+0.00045769	test-auc:0.726819+0.00336413
[134]	train-auc:0.798378+0.00051048	test-auc:0.726886+0.00338568
[135]	train-auc:0.79864+0.000557947	test-auc:0.726916+0.00335021
[136]	train-auc:0.79892+0.000550174	test-auc:0.72694+0.00336864
[137]	train-auc:0.799187+0.000532356	test-auc:0.727097+0.00337488
[138]	train-auc:0.799348+0.000591361	test-auc:0.72712+0.003313
[139]	train-auc:0.799697+0.000604442	test-auc:0.727201+0.0032966
[140]	train-auc:0.799985+0.000624592	test-auc:0.727269+0.00338006
[141]	train-auc:0.800196+0.000626874	test-auc:0.727329+0.00341221
[142]	train-auc:0.800373+0.000714139	test-auc:0.727436+0.00346664
[143]	train-auc:0.80074+

[255]	train-auc:0.830618+0.00123859	test-auc:0.732893+0.00236691
[256]	train-auc:0.830904+0.00118788	test-auc:0.732985+0.0023571
[257]	train-auc:0.831213+0.00113365	test-auc:0.732994+0.00238573
[258]	train-auc:0.831463+0.00112218	test-auc:0.733057+0.00238367
[259]	train-auc:0.831757+0.00115885	test-auc:0.733163+0.00239743
[260]	train-auc:0.831993+0.00116329	test-auc:0.733219+0.00242664
[261]	train-auc:0.832296+0.00112615	test-auc:0.733263+0.00245268
[262]	train-auc:0.832549+0.00108306	test-auc:0.733306+0.00241099
[263]	train-auc:0.832776+0.00112008	test-auc:0.733319+0.00235409
[264]	train-auc:0.833033+0.001152	test-auc:0.733346+0.00237242
[265]	train-auc:0.833245+0.00114345	test-auc:0.73336+0.0024052
[266]	train-auc:0.833521+0.00116672	test-auc:0.73341+0.00236513
[267]	train-auc:0.833789+0.00114801	test-auc:0.733462+0.00236034
[268]	train-auc:0.83402+0.00116403	test-auc:0.733517+0.00231972
[269]	train-auc:0.834191+0.0011511	test-auc:0.733599+0.00230936
[270]	train-auc:0.834476+0.001149

[382]	train-auc:0.859689+0.000877892	test-auc:0.737494+0.00150998
[383]	train-auc:0.859882+0.00086871	test-auc:0.73751+0.00152336
[384]	train-auc:0.860058+0.000882973	test-auc:0.737532+0.00150072
[385]	train-auc:0.860222+0.000905843	test-auc:0.73753+0.00149192
[386]	train-auc:0.860456+0.000874606	test-auc:0.737551+0.00149668
[387]	train-auc:0.860617+0.000885283	test-auc:0.737568+0.00146264
[388]	train-auc:0.860878+0.000883595	test-auc:0.737566+0.00146244
[389]	train-auc:0.861078+0.000932432	test-auc:0.737571+0.00150795
[390]	train-auc:0.861326+0.000931766	test-auc:0.737638+0.00148899
[391]	train-auc:0.861513+0.000934567	test-auc:0.737659+0.00152348
[392]	train-auc:0.861682+0.000869956	test-auc:0.737703+0.00152303
[393]	train-auc:0.861939+0.000905134	test-auc:0.737715+0.00150983
[394]	train-auc:0.862124+0.000948831	test-auc:0.737765+0.00148492
[395]	train-auc:0.862291+0.000972013	test-auc:0.737798+0.00147321
[396]	train-auc:0.86253+0.000954048	test-auc:0.737809+0.00146319
[397]	train-au

[507]	train-auc:0.881501+0.000929063	test-auc:0.739883+0.000763206
[508]	train-auc:0.881649+0.000904821	test-auc:0.739919+0.000715628
[509]	train-auc:0.881808+0.000874681	test-auc:0.739925+0.000708809
[510]	train-auc:0.88199+0.000847375	test-auc:0.7399+0.000712097
[511]	train-auc:0.882158+0.000848079	test-auc:0.739952+0.000718262
[512]	train-auc:0.882357+0.000838034	test-auc:0.739945+0.000702437
[513]	train-auc:0.882561+0.000847535	test-auc:0.739957+0.000700221
[514]	train-auc:0.882719+0.000910157	test-auc:0.739966+0.000701228
[515]	train-auc:0.882854+0.000872595	test-auc:0.739975+0.000719544
[516]	train-auc:0.882975+0.000905455	test-auc:0.740022+0.000734938
[517]	train-auc:0.883104+0.00089501	test-auc:0.74003+0.000729968
[518]	train-auc:0.883245+0.000935562	test-auc:0.740058+0.000728571
[519]	train-auc:0.883415+0.000928458	test-auc:0.740084+0.000716496
[520]	train-auc:0.883514+0.000952341	test-auc:0.740099+0.000739498
[521]	train-auc:0.883687+0.000996622	test-auc:0.740141+0.000760725


[632]	train-auc:0.8995+0.00100383	test-auc:0.74172+0.0008148
[633]	train-auc:0.899629+0.0010147	test-auc:0.74173+0.000813999
[634]	train-auc:0.899781+0.0010044	test-auc:0.741742+0.000805883
[635]	train-auc:0.899892+0.000999879	test-auc:0.741742+0.000824962
[636]	train-auc:0.900042+0.00102681	test-auc:0.741759+0.00081921
[637]	train-auc:0.900164+0.0010663	test-auc:0.74176+0.000826383
[638]	train-auc:0.900299+0.00100402	test-auc:0.741775+0.000829105
[639]	train-auc:0.900427+0.00098062	test-auc:0.741782+0.000817563
[640]	train-auc:0.900544+0.000951055	test-auc:0.741788+0.000791058
[641]	train-auc:0.900655+0.000910605	test-auc:0.741835+0.000804803
[642]	train-auc:0.900807+0.000888063	test-auc:0.741816+0.000802656
[643]	train-auc:0.90097+0.000921221	test-auc:0.741833+0.000808124
[644]	train-auc:0.901129+0.000929168	test-auc:0.741846+0.000800502
[645]	train-auc:0.901287+0.00091439	test-auc:0.741865+0.000799615
[646]	train-auc:0.901449+0.000939094	test-auc:0.741885+0.0008078
[647]	train-auc:0

[757]	train-auc:0.914533+0.000884136	test-auc:0.742313+0.0012081
[758]	train-auc:0.914668+0.000925273	test-auc:0.742293+0.00123835
[759]	train-auc:0.914813+0.000922823	test-auc:0.742294+0.00123276
[760]	train-auc:0.914927+0.000880189	test-auc:0.74228+0.00121788
[761]	train-auc:0.915093+0.000839677	test-auc:0.742311+0.00121892
[762]	train-auc:0.91522+0.000825795	test-auc:0.742328+0.00122568
[763]	train-auc:0.915323+0.000878834	test-auc:0.742331+0.00123158
[764]	train-auc:0.915406+0.00084935	test-auc:0.742314+0.0012092
[765]	train-auc:0.915529+0.000826537	test-auc:0.74232+0.00120124
[766]	train-auc:0.915635+0.000843353	test-auc:0.742334+0.00120144
[767]	train-auc:0.915755+0.000809289	test-auc:0.742312+0.00120575
[768]	train-auc:0.915853+0.000792853	test-auc:0.742289+0.00123536
[769]	train-auc:0.915968+0.000823914	test-auc:0.742298+0.00124955
[770]	train-auc:0.916088+0.000772339	test-auc:0.742306+0.00125975
[771]	train-auc:0.91618+0.000779207	test-auc:0.742309+0.00127034
[772]	train-auc:0

[882]	train-auc:0.927466+0.000799502	test-auc:0.742633+0.00146206
[883]	train-auc:0.927546+0.000790128	test-auc:0.742624+0.0014638
[884]	train-auc:0.927675+0.000830042	test-auc:0.742649+0.0014828
[885]	train-auc:0.927769+0.000855244	test-auc:0.74266+0.0014805
[886]	train-auc:0.927888+0.000869294	test-auc:0.742688+0.0014656
[887]	train-auc:0.927972+0.000890229	test-auc:0.742683+0.00146253
[888]	train-auc:0.928061+0.000885513	test-auc:0.742665+0.0014829
[889]	train-auc:0.92815+0.000914497	test-auc:0.742659+0.00147543
[890]	train-auc:0.928265+0.0009041	test-auc:0.742658+0.00146749
[891]	train-auc:0.928348+0.000880767	test-auc:0.742659+0.00147314
[892]	train-auc:0.928434+0.000865264	test-auc:0.742685+0.0014828
[893]	train-auc:0.928533+0.000860754	test-auc:0.742648+0.00146319
[894]	train-auc:0.92865+0.000897407	test-auc:0.742659+0.00145153
[895]	train-auc:0.928733+0.000892853	test-auc:0.742652+0.00143776
[896]	train-auc:0.92885+0.000903465	test-auc:0.742651+0.00145174
[897]	train-auc:0.9289

### LGBM w/ new features

In [170]:
params_gen = {
    'objective':'binary',
    'seed': 42,
    
    'learning_rate': 0.009,
    'num_threads': 3,
    'metric': 'auc',
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 1e-3,
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq': 1,
    'num_leaves': 38
}

lgb_train_gen = lgb.Dataset(new_train[gen_features], label=new_train.target, free_raw_data=False)

In [144]:
from itertools import combinations
f_combos = [neofeatures] + [[x1, x2] for x1, x2 in combinations(neofeatures, 2)]+ [x for x in neofeatures]

In [147]:
train_results = {}
for f_set in f_combos:
    print('{} training...'.format(f_set))
    temp_features = n_features
    if type(f_set) == list:
        for f in f_set:
            temp_features.append(f)
        f_key = '_'.join(f_set)
    else:
        temp_features.append(f_set)
        f_key = f_set
    temp_train_set = lgb.Dataset(new_train[temp_features], label=new_train.target, free_raw_data=False)
    temp_result = lgb.cv(
        params_gen, 
        temp_train_set, 
        n_rounds, 
        folds=cv.split(new_train[temp_features], new_train.target),
        early_stopping_rounds=100, 
        verbose_eval=1)
    train_results[f_key] = temp_result['auc-mean'][-1]

['338or339', '329*338', '144or338'] training...
[1]	cv_agg's auc: 0.684023 + 0.00949989
[2]	cv_agg's auc: 0.698122 + 0.00582416
[3]	cv_agg's auc: 0.701011 + 0.00593267
[4]	cv_agg's auc: 0.702943 + 0.00600607
[5]	cv_agg's auc: 0.704493 + 0.00693391
[6]	cv_agg's auc: 0.706758 + 0.0064309
[7]	cv_agg's auc: 0.708644 + 0.00677011
[8]	cv_agg's auc: 0.708264 + 0.00716005
[9]	cv_agg's auc: 0.708323 + 0.00680817
[10]	cv_agg's auc: 0.708216 + 0.00680153
[11]	cv_agg's auc: 0.708582 + 0.00688741
[12]	cv_agg's auc: 0.708799 + 0.0069087
[13]	cv_agg's auc: 0.709147 + 0.00753729
[14]	cv_agg's auc: 0.709615 + 0.00773267
[15]	cv_agg's auc: 0.710134 + 0.00791628
[16]	cv_agg's auc: 0.710325 + 0.00815875
[17]	cv_agg's auc: 0.710415 + 0.00816101
[18]	cv_agg's auc: 0.710473 + 0.00833572
[19]	cv_agg's auc: 0.710543 + 0.00806524
[20]	cv_agg's auc: 0.710741 + 0.00798776
[21]	cv_agg's auc: 0.710964 + 0.0082825
[22]	cv_agg's auc: 0.71105 + 0.00828784
[23]	cv_agg's auc: 0.71128 + 0.00805202
[24]	cv_agg's auc: 0.71

[199]	cv_agg's auc: 0.729474 + 0.00592401
[200]	cv_agg's auc: 0.729555 + 0.00587976
[201]	cv_agg's auc: 0.72961 + 0.00587125
[202]	cv_agg's auc: 0.729635 + 0.00587197
[203]	cv_agg's auc: 0.729696 + 0.00584217
[204]	cv_agg's auc: 0.729751 + 0.00580581
[205]	cv_agg's auc: 0.72983 + 0.0058116
[206]	cv_agg's auc: 0.729818 + 0.00581089
[207]	cv_agg's auc: 0.729857 + 0.00582852
[208]	cv_agg's auc: 0.729964 + 0.0058186
[209]	cv_agg's auc: 0.730061 + 0.00577199
[210]	cv_agg's auc: 0.7301 + 0.00580088
[211]	cv_agg's auc: 0.730195 + 0.00582788
[212]	cv_agg's auc: 0.730239 + 0.00584108
[213]	cv_agg's auc: 0.730302 + 0.00582962
[214]	cv_agg's auc: 0.730372 + 0.00582454
[215]	cv_agg's auc: 0.730409 + 0.00574867
[216]	cv_agg's auc: 0.730461 + 0.00580577
[217]	cv_agg's auc: 0.730498 + 0.00586347
[218]	cv_agg's auc: 0.730546 + 0.00588367
[219]	cv_agg's auc: 0.730599 + 0.00585815
[220]	cv_agg's auc: 0.730624 + 0.00585625
[221]	cv_agg's auc: 0.730703 + 0.00586173
[222]	cv_agg's auc: 0.730753 + 0.0058573

[397]	cv_agg's auc: 0.738989 + 0.00469508
[398]	cv_agg's auc: 0.739007 + 0.00469752
[399]	cv_agg's auc: 0.739045 + 0.00469583
[400]	cv_agg's auc: 0.73907 + 0.0046893
[401]	cv_agg's auc: 0.73912 + 0.00465258
[402]	cv_agg's auc: 0.739167 + 0.0046578
[403]	cv_agg's auc: 0.73922 + 0.00465429
[404]	cv_agg's auc: 0.739256 + 0.00464715
[405]	cv_agg's auc: 0.739319 + 0.00465917
[406]	cv_agg's auc: 0.739348 + 0.00464938
[407]	cv_agg's auc: 0.739369 + 0.00465847
[408]	cv_agg's auc: 0.739371 + 0.0046252
[409]	cv_agg's auc: 0.739378 + 0.0046068
[410]	cv_agg's auc: 0.739422 + 0.00459627
[411]	cv_agg's auc: 0.739466 + 0.00459745
[412]	cv_agg's auc: 0.739512 + 0.00460073
[413]	cv_agg's auc: 0.73951 + 0.0045696
[414]	cv_agg's auc: 0.739557 + 0.00456949
[415]	cv_agg's auc: 0.739557 + 0.00458224
[416]	cv_agg's auc: 0.739571 + 0.00456348
[417]	cv_agg's auc: 0.73956 + 0.00457461
[418]	cv_agg's auc: 0.739607 + 0.00460128
[419]	cv_agg's auc: 0.73963 + 0.00457415
[420]	cv_agg's auc: 0.739642 + 0.00455819
[42

[595]	cv_agg's auc: 0.74322 + 0.00423606
[596]	cv_agg's auc: 0.743235 + 0.00424791
[597]	cv_agg's auc: 0.743252 + 0.00423714
[598]	cv_agg's auc: 0.743268 + 0.00425887
[599]	cv_agg's auc: 0.743287 + 0.00425785
[600]	cv_agg's auc: 0.743284 + 0.00427612
[601]	cv_agg's auc: 0.743306 + 0.00428912
[602]	cv_agg's auc: 0.743331 + 0.00428296
[603]	cv_agg's auc: 0.743361 + 0.00428296
[604]	cv_agg's auc: 0.743382 + 0.00426892
[605]	cv_agg's auc: 0.743404 + 0.00422361
[606]	cv_agg's auc: 0.743423 + 0.00421074
[607]	cv_agg's auc: 0.743449 + 0.0042037
[608]	cv_agg's auc: 0.743474 + 0.00420865
[609]	cv_agg's auc: 0.74351 + 0.0042164
[610]	cv_agg's auc: 0.743507 + 0.00421372
[611]	cv_agg's auc: 0.74354 + 0.00420358
[612]	cv_agg's auc: 0.743577 + 0.00420231
[613]	cv_agg's auc: 0.743554 + 0.00416721
[614]	cv_agg's auc: 0.743575 + 0.00415766
[615]	cv_agg's auc: 0.743598 + 0.00414333
[616]	cv_agg's auc: 0.743595 + 0.00415862
[617]	cv_agg's auc: 0.743596 + 0.00417709
[618]	cv_agg's auc: 0.743581 + 0.004165

[791]	cv_agg's auc: 0.745096 + 0.00391573
[792]	cv_agg's auc: 0.745092 + 0.00389077
[793]	cv_agg's auc: 0.74508 + 0.00387563
[794]	cv_agg's auc: 0.745104 + 0.00386178
[795]	cv_agg's auc: 0.745087 + 0.00387098
[796]	cv_agg's auc: 0.745104 + 0.00385391
[797]	cv_agg's auc: 0.745098 + 0.0038685
[798]	cv_agg's auc: 0.74511 + 0.00384635
[799]	cv_agg's auc: 0.74511 + 0.00384492
[800]	cv_agg's auc: 0.745123 + 0.00384158
[801]	cv_agg's auc: 0.745144 + 0.00385815
[802]	cv_agg's auc: 0.745136 + 0.00384374
[803]	cv_agg's auc: 0.745124 + 0.003842
[804]	cv_agg's auc: 0.745176 + 0.00384786
[805]	cv_agg's auc: 0.745196 + 0.00384629
[806]	cv_agg's auc: 0.745198 + 0.00386046
[807]	cv_agg's auc: 0.745221 + 0.00384727
[808]	cv_agg's auc: 0.745227 + 0.00382795
[809]	cv_agg's auc: 0.745229 + 0.00382568
[810]	cv_agg's auc: 0.745252 + 0.00383305
[811]	cv_agg's auc: 0.745262 + 0.00383864
[812]	cv_agg's auc: 0.745267 + 0.003831
[813]	cv_agg's auc: 0.745269 + 0.00382353
[814]	cv_agg's auc: 0.745276 + 0.00381549


...

[581]	cv_agg's auc: 0.742828 + 0.00419193
[582]	cv_agg's auc: 0.742816 + 0.00418852
[583]	cv_agg's auc: 0.742843 + 0.00418651
[584]	cv_agg's auc: 0.742842 + 0.00420039
[585]	cv_agg's auc: 0.742851 + 0.00419642
[586]	cv_agg's auc: 0.742845 + 0.00420807
[587]	cv_agg's auc: 0.742865 + 0.00419833
[588]	cv_agg's auc: 0.742861 + 0.00421098
[589]	cv_agg's auc: 0.742873 + 0.0042088
[590]	cv_agg's auc: 0.74287 + 0.00417926
[591]	cv_agg's auc: 0.742895 + 0.00415277
[592]	cv_agg's auc: 0.742918 + 0.00415515
[593]	cv_agg's auc: 0.742909 + 0.00416396
[594]	cv_agg's auc: 0.742938 + 0.00414084
[595]	cv_agg's auc: 0.742972 + 0.00413427
[596]	cv_agg's auc: 0.743005 + 0.00412732
[597]	cv_agg's auc: 0.743019 + 0.00415159
[598]	cv_agg's auc: 0.743022 + 0.00416595
[599]	cv_agg's auc: 0.74306 + 0.00417031
[600]	cv_agg's auc: 0.743046 + 0.00415713
[601]	cv_agg's auc: 0.743103 + 0.00413454
[602]	cv_agg's auc: 0.743128 + 0.00416393
[603]	cv_agg's auc: 0.743123 + 0.00415638
[604]	cv_agg's auc: 0.743111 + 0.0041

[779]	cv_agg's auc: 0.744786 + 0.00407559
[780]	cv_agg's auc: 0.744783 + 0.00410201
[781]	cv_agg's auc: 0.744815 + 0.00409932
[782]	cv_agg's auc: 0.744796 + 0.00409457
[783]	cv_agg's auc: 0.744802 + 0.00407281
[784]	cv_agg's auc: 0.744791 + 0.00407377
[785]	cv_agg's auc: 0.744778 + 0.00408385
[786]	cv_agg's auc: 0.744764 + 0.00406266
[787]	cv_agg's auc: 0.744758 + 0.0040571
[788]	cv_agg's auc: 0.744765 + 0.00407734
[789]	cv_agg's auc: 0.744766 + 0.00410489
[790]	cv_agg's auc: 0.74477 + 0.0040968
[791]	cv_agg's auc: 0.744772 + 0.00408839
[792]	cv_agg's auc: 0.744785 + 0.00409934
[793]	cv_agg's auc: 0.744774 + 0.00407883
[794]	cv_agg's auc: 0.744768 + 0.00408846
[795]	cv_agg's auc: 0.744775 + 0.00410276
[796]	cv_agg's auc: 0.744784 + 0.00408449
[797]	cv_agg's auc: 0.744782 + 0.00410692
[798]	cv_agg's auc: 0.744783 + 0.00409939
[799]	cv_agg's auc: 0.744795 + 0.00408759
[800]	cv_agg's auc: 0.744808 + 0.00408094
[801]	cv_agg's auc: 0.744795 + 0.00410843
[802]	cv_agg's auc: 0.744814 + 0.0041

[976]	cv_agg's auc: 0.745376 + 0.00423378
[977]	cv_agg's auc: 0.745401 + 0.00421045
[978]	cv_agg's auc: 0.745398 + 0.00420794
[979]	cv_agg's auc: 0.745413 + 0.00420038
[980]	cv_agg's auc: 0.745413 + 0.00419685
[981]	cv_agg's auc: 0.745423 + 0.00421133
[982]	cv_agg's auc: 0.745447 + 0.00423482
[983]	cv_agg's auc: 0.745446 + 0.00425111
[984]	cv_agg's auc: 0.745452 + 0.00426471
[985]	cv_agg's auc: 0.74542 + 0.00424421
[986]	cv_agg's auc: 0.745428 + 0.00423996
[987]	cv_agg's auc: 0.745459 + 0.00424764
[988]	cv_agg's auc: 0.745466 + 0.00424274
[989]	cv_agg's auc: 0.745467 + 0.00426197
[990]	cv_agg's auc: 0.745448 + 0.00421766
[991]	cv_agg's auc: 0.745432 + 0.0042026
[992]	cv_agg's auc: 0.745447 + 0.00421656
[993]	cv_agg's auc: 0.745477 + 0.00422901
[994]	cv_agg's auc: 0.74546 + 0.00423917
[995]	cv_agg's auc: 0.74545 + 0.00423614
[996]	cv_agg's auc: 0.745446 + 0.00421442
[997]	cv_agg's auc: 0.745459 + 0.0042409
[998]	cv_agg's auc: 0.745474 + 0.0042559
[999]	cv_agg's auc: 0.745471 + 0.0042692

[1169]	cv_agg's auc: 0.746017 + 0.00421696
[1170]	cv_agg's auc: 0.746024 + 0.00421009
[1171]	cv_agg's auc: 0.746029 + 0.00421775
[1172]	cv_agg's auc: 0.746033 + 0.00422259
[1173]	cv_agg's auc: 0.74602 + 0.00424028
[1174]	cv_agg's auc: 0.746017 + 0.0042407
[1175]	cv_agg's auc: 0.74603 + 0.0042465
[1176]	cv_agg's auc: 0.746016 + 0.00426819
[1177]	cv_agg's auc: 0.746023 + 0.00427882
[1178]	cv_agg's auc: 0.746025 + 0.00426372
[1179]	cv_agg's auc: 0.746018 + 0.00426515
[1180]	cv_agg's auc: 0.746014 + 0.00424024
[1181]	cv_agg's auc: 0.746016 + 0.00425406
[1182]	cv_agg's auc: 0.745996 + 0.00425087
[1183]	cv_agg's auc: 0.746011 + 0.00425395
[1184]	cv_agg's auc: 0.746024 + 0.00423919
[1185]	cv_agg's auc: 0.746019 + 0.00424487
[1186]	cv_agg's auc: 0.746038 + 0.00427373
[1187]	cv_agg's auc: 0.746034 + 0.0042773
[1188]	cv_agg's auc: 0.746038 + 0.00427475
[1189]	cv_agg's auc: 0.746034 + 0.00429638
[1190]	cv_agg's auc: 0.746038 + 0.00429493
[1191]	cv_agg's auc: 0.746046 + 0.00430888
[1192]	cv_agg's 

[1362]	cv_agg's auc: 0.74633 + 0.00433082
[1363]	cv_agg's auc: 0.746322 + 0.00433688
[1364]	cv_agg's auc: 0.746304 + 0.00433904
[1365]	cv_agg's auc: 0.746297 + 0.00434645
[1366]	cv_agg's auc: 0.746294 + 0.00436102
[1367]	cv_agg's auc: 0.746294 + 0.00436639
[1368]	cv_agg's auc: 0.746251 + 0.00437222
[1369]	cv_agg's auc: 0.74626 + 0.0043668
[1370]	cv_agg's auc: 0.746248 + 0.00438091
[1371]	cv_agg's auc: 0.746253 + 0.00438009
[1372]	cv_agg's auc: 0.746281 + 0.00438379
[1373]	cv_agg's auc: 0.7463 + 0.00438214
[1374]	cv_agg's auc: 0.746292 + 0.0043809
[1375]	cv_agg's auc: 0.746287 + 0.00439943
[1376]	cv_agg's auc: 0.746291 + 0.00438934
[1377]	cv_agg's auc: 0.746268 + 0.00439766
[1378]	cv_agg's auc: 0.746262 + 0.00440072
[1379]	cv_agg's auc: 0.746259 + 0.00438397
[1380]	cv_agg's auc: 0.746253 + 0.00438487
[1381]	cv_agg's auc: 0.746241 + 0.00438182
[1382]	cv_agg's auc: 0.746241 + 0.00438803
[1383]	cv_agg's auc: 0.746239 + 0.00439811
[1384]	cv_agg's auc: 0.746245 + 0.00441664
[1385]	cv_agg's a

In [154]:
best_set = max(train_results, key=train_results.get).split('_')

In [171]:
lgb.cv(
        params_gen, 
        lgb_train_gen, 
        n_rounds, 
        folds=cv.split(new_train[n_features+best_set], new_train.target),
        early_stopping_rounds=100, 
        verbose_eval=1
)

[1]	cv_agg's auc: 0.68661 + 0.00911873
[2]	cv_agg's auc: 0.697844 + 0.00668412
[3]	cv_agg's auc: 0.70196 + 0.00533057
[4]	cv_agg's auc: 0.705159 + 0.00633155
[5]	cv_agg's auc: 0.706965 + 0.00653977
[6]	cv_agg's auc: 0.708671 + 0.00735019
[7]	cv_agg's auc: 0.709237 + 0.00640299
[8]	cv_agg's auc: 0.710255 + 0.00615239
[9]	cv_agg's auc: 0.710771 + 0.00600015
[10]	cv_agg's auc: 0.711248 + 0.00606335
[11]	cv_agg's auc: 0.711313 + 0.00617131
[12]	cv_agg's auc: 0.711964 + 0.00553902
[13]	cv_agg's auc: 0.712281 + 0.00627295
[14]	cv_agg's auc: 0.712939 + 0.00630644
[15]	cv_agg's auc: 0.713575 + 0.00626848
[16]	cv_agg's auc: 0.714362 + 0.00632769
[17]	cv_agg's auc: 0.714634 + 0.0065409
[18]	cv_agg's auc: 0.71472 + 0.00676589
[19]	cv_agg's auc: 0.714661 + 0.00683118
[20]	cv_agg's auc: 0.714606 + 0.00699279
[21]	cv_agg's auc: 0.71478 + 0.00671705
[22]	cv_agg's auc: 0.714985 + 0.00663953
[23]	cv_agg's auc: 0.715203 + 0.00672468
[24]	cv_agg's auc: 0.715142 + 0.00709495
[25]	cv_agg's auc: 0.715553 + 

[201]	cv_agg's auc: 0.731314 + 0.00623984
[202]	cv_agg's auc: 0.731343 + 0.00622575
[203]	cv_agg's auc: 0.731414 + 0.0062263
[204]	cv_agg's auc: 0.731449 + 0.00619628
[205]	cv_agg's auc: 0.731534 + 0.00620658
[206]	cv_agg's auc: 0.731553 + 0.00623691
[207]	cv_agg's auc: 0.731604 + 0.00623457
[208]	cv_agg's auc: 0.731673 + 0.00625499
[209]	cv_agg's auc: 0.731707 + 0.0062795
[210]	cv_agg's auc: 0.731777 + 0.00631396
[211]	cv_agg's auc: 0.731874 + 0.00627369
[212]	cv_agg's auc: 0.731968 + 0.00626072
[213]	cv_agg's auc: 0.732015 + 0.00621761
[214]	cv_agg's auc: 0.732067 + 0.00623732
[215]	cv_agg's auc: 0.732143 + 0.00618268
[216]	cv_agg's auc: 0.73222 + 0.00619338
[217]	cv_agg's auc: 0.73221 + 0.00617835
[218]	cv_agg's auc: 0.732317 + 0.00619234
[219]	cv_agg's auc: 0.732418 + 0.00622837
[220]	cv_agg's auc: 0.732479 + 0.00628424
[221]	cv_agg's auc: 0.732518 + 0.00628108
[222]	cv_agg's auc: 0.732582 + 0.00631849
[223]	cv_agg's auc: 0.732676 + 0.00628621
[224]	cv_agg's auc: 0.732782 + 0.00625

[399]	cv_agg's auc: 0.740836 + 0.00525657
[400]	cv_agg's auc: 0.740847 + 0.00525627
[401]	cv_agg's auc: 0.740945 + 0.00525812
[402]	cv_agg's auc: 0.740966 + 0.00524551
[403]	cv_agg's auc: 0.740985 + 0.00521265
[404]	cv_agg's auc: 0.741019 + 0.00522087
[405]	cv_agg's auc: 0.74108 + 0.00524341
[406]	cv_agg's auc: 0.741071 + 0.00522965
[407]	cv_agg's auc: 0.74111 + 0.00522165
[408]	cv_agg's auc: 0.741121 + 0.00519932
[409]	cv_agg's auc: 0.741143 + 0.00520046
[410]	cv_agg's auc: 0.741182 + 0.00519119
[411]	cv_agg's auc: 0.741204 + 0.005193
[412]	cv_agg's auc: 0.741226 + 0.00520095
[413]	cv_agg's auc: 0.741256 + 0.00520115
[414]	cv_agg's auc: 0.741275 + 0.00519517
[415]	cv_agg's auc: 0.741249 + 0.00518435
[416]	cv_agg's auc: 0.741274 + 0.00516305
[417]	cv_agg's auc: 0.741309 + 0.00520529
[418]	cv_agg's auc: 0.741376 + 0.0052291
[419]	cv_agg's auc: 0.741401 + 0.00520175
[420]	cv_agg's auc: 0.74142 + 0.00517283
[421]	cv_agg's auc: 0.741469 + 0.0051532
[422]	cv_agg's auc: 0.741463 + 0.00520453

[595]	cv_agg's auc: 0.744618 + 0.00483601
[596]	cv_agg's auc: 0.744628 + 0.00484147
[597]	cv_agg's auc: 0.744629 + 0.00483377
[598]	cv_agg's auc: 0.744638 + 0.00484203
[599]	cv_agg's auc: 0.744662 + 0.00484466
[600]	cv_agg's auc: 0.744655 + 0.00482635
[601]	cv_agg's auc: 0.744691 + 0.00480406
[602]	cv_agg's auc: 0.744713 + 0.00479884
[603]	cv_agg's auc: 0.744755 + 0.00479498
[604]	cv_agg's auc: 0.744785 + 0.00479556
[605]	cv_agg's auc: 0.744803 + 0.00479352
[606]	cv_agg's auc: 0.744801 + 0.00478236
[607]	cv_agg's auc: 0.744815 + 0.00478875
[608]	cv_agg's auc: 0.744829 + 0.0047811
[609]	cv_agg's auc: 0.744878 + 0.00480024
[610]	cv_agg's auc: 0.744908 + 0.0047992
[611]	cv_agg's auc: 0.744899 + 0.00481201
[612]	cv_agg's auc: 0.744922 + 0.00479051
[613]	cv_agg's auc: 0.744949 + 0.00478464
[614]	cv_agg's auc: 0.744952 + 0.00477964
[615]	cv_agg's auc: 0.744975 + 0.00475746
[616]	cv_agg's auc: 0.745004 + 0.00476521
[617]	cv_agg's auc: 0.745018 + 0.00474269
[618]	cv_agg's auc: 0.745066 + 0.004

[792]	cv_agg's auc: 0.746135 + 0.00448261
[793]	cv_agg's auc: 0.746141 + 0.00446707
[794]	cv_agg's auc: 0.746138 + 0.00446546
[795]	cv_agg's auc: 0.746143 + 0.00445821
[796]	cv_agg's auc: 0.746151 + 0.00448717
[797]	cv_agg's auc: 0.746151 + 0.00447043
[798]	cv_agg's auc: 0.746163 + 0.00445188
[799]	cv_agg's auc: 0.746138 + 0.0044421
[800]	cv_agg's auc: 0.746175 + 0.00446748
[801]	cv_agg's auc: 0.746166 + 0.00448187
[802]	cv_agg's auc: 0.746171 + 0.00446783
[803]	cv_agg's auc: 0.746159 + 0.00445375
[804]	cv_agg's auc: 0.746169 + 0.00441901
[805]	cv_agg's auc: 0.746185 + 0.00440231
[806]	cv_agg's auc: 0.746216 + 0.00441737
[807]	cv_agg's auc: 0.746215 + 0.00442191
[808]	cv_agg's auc: 0.746216 + 0.0044217
[809]	cv_agg's auc: 0.746233 + 0.00440833
[810]	cv_agg's auc: 0.74625 + 0.00442517
[811]	cv_agg's auc: 0.746266 + 0.0044023
[812]	cv_agg's auc: 0.746287 + 0.00441302
[813]	cv_agg's auc: 0.746291 + 0.00441311
[814]	cv_agg's auc: 0.746318 + 0.00444923
[815]	cv_agg's auc: 0.746345 + 0.00442

[988]	cv_agg's auc: 0.746876 + 0.00410506
[989]	cv_agg's auc: 0.746857 + 0.00408868
[990]	cv_agg's auc: 0.746846 + 0.00409232
[991]	cv_agg's auc: 0.746849 + 0.00409167
[992]	cv_agg's auc: 0.746846 + 0.00410362
[993]	cv_agg's auc: 0.746831 + 0.00409752
[994]	cv_agg's auc: 0.746819 + 0.00409386
[995]	cv_agg's auc: 0.746812 + 0.00409855
[996]	cv_agg's auc: 0.746823 + 0.00411399
[997]	cv_agg's auc: 0.746814 + 0.00410927
[998]	cv_agg's auc: 0.746826 + 0.00411573
[999]	cv_agg's auc: 0.746848 + 0.00412106
[1000]	cv_agg's auc: 0.746839 + 0.00416154
[1001]	cv_agg's auc: 0.746842 + 0.00413778
[1002]	cv_agg's auc: 0.746833 + 0.00413695
[1003]	cv_agg's auc: 0.746821 + 0.00413882
[1004]	cv_agg's auc: 0.746834 + 0.00414891
[1005]	cv_agg's auc: 0.746821 + 0.00414546
[1006]	cv_agg's auc: 0.746814 + 0.00416571
[1007]	cv_agg's auc: 0.746816 + 0.00416646
[1008]	cv_agg's auc: 0.746834 + 0.00416034
[1009]	cv_agg's auc: 0.74683 + 0.00417769
[1010]	cv_agg's auc: 0.746816 + 0.00417016
[1011]	cv_agg's auc: 0.7

[1181]	cv_agg's auc: 0.746981 + 0.00429185
[1182]	cv_agg's auc: 0.746956 + 0.00429917
[1183]	cv_agg's auc: 0.746947 + 0.00430901
[1184]	cv_agg's auc: 0.74696 + 0.00429437
[1185]	cv_agg's auc: 0.746972 + 0.00429244
[1186]	cv_agg's auc: 0.746985 + 0.00429373
[1187]	cv_agg's auc: 0.746973 + 0.00428501
[1188]	cv_agg's auc: 0.746978 + 0.00427247
[1189]	cv_agg's auc: 0.746986 + 0.00426292
[1190]	cv_agg's auc: 0.746964 + 0.00427474
[1191]	cv_agg's auc: 0.746973 + 0.0042778
[1192]	cv_agg's auc: 0.746961 + 0.00426125
[1193]	cv_agg's auc: 0.746973 + 0.00426761
[1194]	cv_agg's auc: 0.746955 + 0.00424857
[1195]	cv_agg's auc: 0.746957 + 0.00424949
[1196]	cv_agg's auc: 0.74695 + 0.00424281
[1197]	cv_agg's auc: 0.746964 + 0.00427605
[1198]	cv_agg's auc: 0.746976 + 0.00426881
[1199]	cv_agg's auc: 0.746956 + 0.00428288
[1200]	cv_agg's auc: 0.746974 + 0.00428868
[1201]	cv_agg's auc: 0.74697 + 0.00427003
[1202]	cv_agg's auc: 0.746969 + 0.00428018
[1203]	cv_agg's auc: 0.746954 + 0.0042924
[1204]	cv_agg's 

{'auc-mean': [0.6866102824780602,
  0.6978436532628157,
  0.7019601661108061,
  0.7051593703198115,
  0.7069647606068404,
  0.7086708249157448,
  0.7092366332273063,
  0.7102547933194485,
  0.7107711591148244,
  0.7112479085749221,
  0.7113125530458704,
  0.7119642307225336,
  0.7122812305313038,
  0.7129386447911543,
  0.7135751393517018,
  0.7143617669499444,
  0.7146342237444383,
  0.7147204303266352,
  0.7146607455591136,
  0.7146061569603692,
  0.7147796953784387,
  0.7149847216283253,
  0.7152030466554176,
  0.7151419359297543,
  0.7155531855522084,
  0.715601896223832,
  0.715967247049843,
  0.716189419312232,
  0.716347002774421,
  0.7163511177646761,
  0.7165133517617672,
  0.7166919403191028,
  0.7168248266353554,
  0.7170423900086536,
  0.7172453976316882,
  0.7174081444837701,
  0.7173579114085253,
  0.7174625370866059,
  0.7176315858112414,
  0.7178533928912838,
  0.7179062463083566,
  0.718073090944709,
  0.7182387757373028,
  0.7184168369724631,
  0.7184178249821183,
  0